In [1]:
%config Completer.use_jedi = False
%load_ext autoreload

# Imports and Loads

In [2]:
from crawler.src import crawler 
from ast import literal_eval

ImportError: cannot import name 'crawler' from 'crawler.src' (/home/msnow/git/pax_tt_recommender/crawler/src/__init__.py)

In [3]:
import importlib
importlib.reload(crawler)

<module 'crawler.src.crawler' from '/home/ubuntu/git/pax_tt_recommender/crawler/src/crawler.py'>

In [2]:
import pandas as pd
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
from sklearn.preprocessing import LabelEncoder
from implicit.als import AlternatingLeastSquares
import csv
import requests
from bs4 import BeautifulSoup

In [3]:
pd.options.display.max_columns=200
pd.options.display.max_rows=200

# Functions

In [53]:
# from bs4 import BeautifulSoup
# import bs4
# import requests
import pandas as pd
# import json
# from requests.compat import urljoin
# from datetime import datetime
# import re
# from time import sleep
# import numpy as np
# from zipfile import ZipFile
# from io import BytesIO
# import math


# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service

In [39]:
def get_driver_and_cookies():
    LOGIN_USERNAME_FIELD = '//*[@id="inputUsername"]'
    LOGIN_PASSWORD_FIELD = '//*[@id="inputPassword"]'
    LOGIN_BUTTON = '//*[@id="mainbody"]/div/div/gg-login-page/div[1]/div/gg-login-form/form/fieldset/div[3]/button[1]'

    with open("/home/msnow/config.json", "r") as fp:
        secrets = json.load(fp)
    USERNAME = secrets["bgg_crawler"]["username"]
    PASSWORD = secrets["bgg_crawler"]["password"]

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    cookies = {}

    driver = webdriver.Chrome(
        service=Service("/usr/lib/chromium-browser/chromedriver"),
        options=chrome_options,
    )
    driver.get("https://boardgamegeek.com/login")
    login = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, LOGIN_USERNAME_FIELD))
    )
    password = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, LOGIN_PASSWORD_FIELD))
    )

    login_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, LOGIN_BUTTON))
    )

    login.send_keys(USERNAME)
    password.send_keys(PASSWORD)

    login_button.click()
    sleep(1)
    selenium_cookies = driver.get_cookies()
    for cookie in selenium_cookies:
        cookies[cookie["name"]] = cookie["value"]
    return cookies

In [102]:
def get_boardgame_list(cookies:dict):
    bg_list_pg_url = "https://boardgamegeek.com/data_dumps/bg_ranks"
    resp = requests.get(bg_list_pg_url, cookies=cookies)
    soup = BeautifulSoup(resp.content, "html.parser")
    bg_list_url = soup.find("div", {"id": "maincontent"})("a")[0]["href"]
    bg_list_zip = requests.get(bg_list_url)
    with ZipFile(BytesIO(bg_list_zip.content)) as archive:
        with archive.open("boardgames_ranks.csv") as csv:
            df_bg_list = pd.read_csv(csv)
            df_bg_list["queried_at_utc"] = datetime.now().replace(microsecond=0).isoformat()
            return df_bg_list

In [238]:
def get_boardgame_raw_data(boardgame_df: pd.DataFrame):
    boardgame_ids = boardgame_df["id"].tolist()
    boardgame_list = []
    batch_size = 3
    for batch_num in range(math.ceil(len(boardgame_ids) / batch_size)):
        batch_ids = boardgame_ids[batch_num * batch_size : (batch_num + 1) * batch_size]
        batch_ids = [str(x) for x in batch_ids]
        bg_info_url = f"https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame&stats=1&versions=1&ratingcomments=1&pagesize=100&page=1&id={','.join(batch_ids)}"
        bgg_response = requests.get(bg_info_url)
        soup_xml = BeautifulSoup(bgg_response.content, "xml")
        games_xml_list = soup_xml.find_all("item", attrs={"type": "boardgame"})
        ratings_count_dict = {}
        for game_xml in games_xml_list:
            game_dict = extract_basic_game_info(game_xml=game_xml)
            game_dict = extract_polls(game_dict=game_dict, game_xml=game_xml)
            game_dict = extract_poll_player_count(
                game_dict=game_dict, game_xml=game_xml
            )
            game_dict = extract_version_info(game_dict=game_dict, game_xml=game_xml)
            if game_dict["numratings"] > 0:
                game_dict = extract_ratings(game_dict=game_dict, game_xml=game_xml)
            ratings_count_dict[game_dict["game_id"]] = game_dict["numratings"]
            max_ratings_page = math.ceil(max(ratings_count_dict.values()) / 100)
            boardgame_list.append(game_dict)
        for page_num in range(2, max_ratings_page):
            # Only grab the pages for games which have enough ratings to be on the page num
            batch_ids_ratings = [
                str(x)
                for x in ratings_count_dict.keys()
                if math.ceil(ratings_count_dict[x] / 100) >= page_num
            ]
            bg_rating_url = f"https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame&ratingcomments=1&pagesize=100&page={page_num}&id={','.join(batch_ids_ratings)}"
            bgg_rating_response = requests.get(bg_rating_url)
            soup_rating_xml = BeautifulSoup(bgg_rating_response.content, "xml")
            ratings_xml_list = soup_rating_xml.find_all(
                "item", attrs={"type": "boardgame"}
            )
            for game_xml in ratings_xml_list:
                game_dict = extract_ratings(game_dict=game_dict, game_xml=game_xml)
        break
    return boardgame_list

In [234]:
def extract_basic_game_info(game_xml: bs4.element.Tag):
    game_dict = {
        "game_id": game_xml["id"],
    }
    if game_xml.find("image") is not None:
        game_dict["thumbnail"] = game_xml.find("thumbnail").text
        game_dict["image"] = game_xml.find("image").text
    values_int = [
        "minplayers",
        "maxplayers",
        "playingtime",
        "minplaytime",
        "maxplaytime",
        "minage",
    ]
    for vals in values_int:
        if game_xml.find(vals) is not None:
            game_dict[vals] = game_xml.find(vals)["value"]
    link_categ = [
        "boardgamecategory",
        "boardgamemechanic",
        "boardgamefamily",
        "boardgameexpansion",
        "boardgameartist",
        "boardgamecompilation",
        "boardgameimplementation",
        "boardgamedesigner",
        "boardgamepublisher",
        "boardgameintegration",
    ]
    for categ in link_categ:
        game_dict[categ] = [
            x["value"] for x in game_xml.find_all("link", {"type": categ})
        ]
    stats_float = ["stddev", "median", "averageweight"]
    for stat in stats_float:
        if game_xml.find(stat) is not None:
            game_dict[stat] = float(game_xml.find(stat)["value"])
    stats_int = [
        "owned",
        "trading",
        "wanting",
        "wishing",
        "numcomments",
        "numweights",
    ]
    for stat in stats_int:
        if game_xml.find(stat) is not None:
            game_dict[stat] = int(game_xml.find(stat)["value"])
    if game_xml.find("comments") is not None:
        game_dict["ratings"] = int(game_xml.find("comments")["totalitems"])
    else:
        game_dict["ratings"] = 0
    return game_dict

In [138]:
def extract_polls(game_dict:dict, game_xml: bs4.element.Tag):
    for poll_name in ["suggested_playerage", "language_dependence"]:
        if poll_name == "suggested_playerage":
            raw_value_col = "value"
        elif poll_name == "language_dependence":
            raw_value_col = "level"
        poll = game_xml.find("poll", attrs={"name": poll_name})
        vote_count = int(poll.attrs["totalvotes"])
        if vote_count > 0:
            result_dict = {"total_votes": vote_count}
            cum_votes = 0
            suggested_prcnt = {}
            suggested_prcnt_col = "value"
            for result_val in poll.find("results").find_all("result"):
                num_votes = int(result_val["numvotes"]) 
                cum_votes += num_votes
                cum_prcnt = round(cum_votes/vote_count*100)
                result_dict[result_val[raw_value_col]] = num_votes
                sugg_prcnt_val = result_val[suggested_prcnt_col]
                if cum_prcnt >= 75:
                    if "75 percent" not in suggested_prcnt:
                        suggested_prcnt["75 percent"] = sugg_prcnt_val
                    if "50 percent" not in suggested_prcnt:
                        suggested_prcnt["50 percent"] = sugg_prcnt_val
                    if "25 percent" not in suggested_prcnt:
                        suggested_prcnt["25 percent"] = sugg_prcnt_val
                elif cum_prcnt >= 50:
                    suggested_prcnt["50 percent"] = sugg_prcnt_val
                    if "25 percent" not in suggested_prcnt:
                        suggested_prcnt["25 percent"] = sugg_prcnt_val
                elif cum_prcnt >= 25:
                    suggested_prcnt["25 percent"] = sugg_prcnt_val
        else:
            result_dict = {}
            suggested_prcnt = {}
        game_dict[poll_name] = result_dict
        game_dict[f"{poll_name}_quartiles"] = suggested_prcnt  
    return game_dict

In [153]:
def extract_poll_player_count(game_dict:dict, game_xml: bs4.element.Tag):
    player_count_poll = game_xml.find("poll", attrs={"name": "suggested_numplayers"})
    result_dict = {"total_votes": int(player_count_poll.attrs["totalvotes"])}
    player_count_results = player_count_poll.find_all("results")
    game_dict["player_count_recs"] = {}
    for player_count in player_count_results:
        num_players = player_count.attrs["numplayers"]
        player_count_values = {
            x.attrs["value"]: int(x.attrs["numvotes"])
            for x in player_count.find_all("result")
        }
        play_count_rec = max(player_count_values, key=player_count_values.get)
        if play_count_rec in game_dict["player_count_recs"]:
            game_dict["player_count_recs"][play_count_rec].append(num_players)
        else:
            game_dict["player_count_recs"][play_count_rec] = [num_players]
        result_dict[num_players] = player_count_values
        result_dict[num_players]["total_votes"] = sum(
            int(x.attrs["numvotes"]) for x in player_count.find_all("result")
        )
    game_dict["suggested_numplayers"] = result_dict  
    return game_dict

In [180]:
def extract_version_info(game_dict: dict, game_xml: bs4.element.Tag):
    version_items = game_xml.find_all("item", attrs={"type": "boardgameversion"})
    version_list = []
    for vrs in version_items:
        version_dict = {
            "version_id": int(vrs["id"]),
            "version_nickname": vrs.find("name", attrs={"type": "primary"})[
                "value"
            ],
            "width": round(float(vrs.find("width")["value"])),
            "length": round(float(vrs.find("length")["value"])),
            "depth": round(float(vrs.find("depth")["value"])),
            "year_published": round(float(vrs.find("yearpublished")["value"])),
            "language": vrs.find("link", attrs={"type": "language"})["value"].lower(),
        }
        if vrs.find("thumbnail") is not None:
            version_dict["thumbnail"] = vrs.find("thumbnail").text
            version_dict["image"] = vrs.find("image").text
        if version_dict["width"] > 0:
            version_list.append(version_dict)
    if len(version_list) > 0:
        game_dict["versions"] = version_list
    return game_dict

In [193]:
def extract_ratings(game_dict: dict, game_xml: bs4.element.Tag):
    ratings_list = game_xml.find_all("comment")
    if "ratings" not in game_dict:
        game_dict["ratings"] = {}
    for rating in ratings_list:
        # round the rating to the nearest 0.5
        rating_round = round(2*float(rating["rating"]))/2
        if rating_round not in game_dict["ratings"]:
            game_dict["ratings"][rating_round] =  [rating["username"]]
        else:
            game_dict["ratings"][rating_round].append(rating["username"])
    return game_dict

In [268]:
def get_boardgame_raw_data(boardgame_df: pd.DataFrame):
    boardgame_ids = boardgame_df["id"].tolist()
    boardgame_master_dict = {}
    batch_size = 3
    for batch_num in range(math.ceil(len(boardgame_ids) / batch_size)):
        batch_ids = boardgame_ids[batch_num * batch_size : (batch_num + 1) * batch_size]
        batch_ids = [str(x) for x in batch_ids]
        bg_info_url = f"https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame&stats=1&versions=1&ratingcomments=1&pagesize=100&page=1&id={','.join(batch_ids)}"
        bgg_response = requests.get(bg_info_url)
        soup_xml = BeautifulSoup(bgg_response.content, "xml")
        games_xml_list = soup_xml.find_all("item", attrs={"type": "boardgame"})
        ratings_count_dict = {}
        for game_xml in games_xml_list:
            game_dict = extract_basic_game_info(game_xml=game_xml)
            game_dict = extract_polls(game_dict=game_dict, game_xml=game_xml)
            game_dict = extract_poll_player_count(
                game_dict=game_dict, game_xml=game_xml
            )
            game_dict = extract_version_info(game_dict=game_dict, game_xml=game_xml)
            if game_dict["numratings"] > 0:
                game_dict = extract_ratings(game_dict=game_dict, game_xml=game_xml)
            ratings_count_dict[game_dict["game_id"]] = game_dict["numratings"]
            max_ratings_page = math.ceil(max(ratings_count_dict.values()) / 100)
            boardgame_master_dict[game_dict["game_id"]] = game_dict
        for page_num in range(2, max_ratings_page + 1):
            print(page_num)
            # Only grab the pages for games which have enough ratings to be on the page num
            batch_ids_ratings = [
                str(x)
                for x in ratings_count_dict.keys()
                if math.ceil(ratings_count_dict[x] / 100) >= page_num
            ]
            bg_rating_url = f"https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame&ratingcomments=1&pagesize=100&page={page_num}&id={','.join(batch_ids_ratings)}"
            bgg_rating_response = requests.get(bg_rating_url)
            soup_rating_xml = BeautifulSoup(bgg_rating_response.content, "xml")
            ratings_xml_list = soup_rating_xml.find_all(
                "item", attrs={"type": "boardgame"}
            )
            for game_xml in ratings_xml_list:
                game_dict = boardgame_master_dict[game_xml["id"]]
                boardgame_master_dict[game_dict["game_id"]] = extract_ratings(
                    game_dict=game_dict, game_xml=game_xml
                )
        break
    return list(boardgame_master_dict.values())


def extract_basic_game_info(game_xml: bs4.element.Tag):
    game_dict = {
        "game_id": game_xml["id"],
    }
    if game_xml.find("image") is not None:
        game_dict["thumbnail"] = game_xml.find("thumbnail").text
        game_dict["image"] = game_xml.find("image").text
    values_int = [
        "minplayers",
        "maxplayers",
        "playingtime",
        "minplaytime",
        "maxplaytime",
        "minage",
    ]
    for vals in values_int:
        if game_xml.find(vals) is not None:
            game_dict[vals] = game_xml.find(vals)["value"]
    link_categ = [
        "boardgamecategory",
        "boardgamemechanic",
        "boardgamefamily",
        "boardgameexpansion",
        "boardgameartist",
        "boardgamecompilation",
        "boardgameimplementation",
        "boardgamedesigner",
        "boardgamepublisher",
        "boardgameintegration",
    ]
    for categ in link_categ:
        game_dict[categ] = [
            x["value"] for x in game_xml.find_all("link", {"type": categ})
        ]
    stats_float = ["stddev", "median", "averageweight"]
    for stat in stats_float:
        if game_xml.find(stat) is not None:
            game_dict[stat] = float(game_xml.find(stat)["value"])
    stats_int = [
        "owned",
        "trading",
        "wanting",
        "wishing",
        "numcomments",
        "numweights",
    ]
    for stat in stats_int:
        if game_xml.find(stat) is not None:
            game_dict[stat] = int(game_xml.find(stat)["value"])
    if game_xml.find("comments") is not None:
        game_dict["numratings"] = int(game_xml.find("comments")["totalitems"])
    else:
        game_dict["numratings"] = 0
    return game_dict


def extract_polls(game_dict: dict, game_xml: bs4.element.Tag):
    for poll_name in ["suggested_playerage", "language_dependence"]:
        if poll_name == "suggested_playerage":
            raw_value_col = "value"
        elif poll_name == "language_dependence":
            raw_value_col = "level"
        poll = game_xml.find("poll", attrs={"name": poll_name})
        vote_count = int(poll.attrs["totalvotes"])
        if vote_count > 0:
            result_dict = {"total_votes": vote_count}
            cum_votes = 0
            suggested_prcnt = {}
            suggested_prcnt_col = "value"
            for result_val in poll.find("results").find_all("result"):
                num_votes = int(result_val["numvotes"])
                cum_votes += num_votes
                cum_prcnt = round(cum_votes / vote_count * 100)
                result_dict[result_val[raw_value_col]] = num_votes
                sugg_prcnt_val = result_val[suggested_prcnt_col]
                if cum_prcnt >= 75:
                    if "75 percent" not in suggested_prcnt:
                        suggested_prcnt["75 percent"] = sugg_prcnt_val
                    if "50 percent" not in suggested_prcnt:
                        suggested_prcnt["50 percent"] = sugg_prcnt_val
                    if "25 percent" not in suggested_prcnt:
                        suggested_prcnt["25 percent"] = sugg_prcnt_val
                elif cum_prcnt >= 50:
                    suggested_prcnt["50 percent"] = sugg_prcnt_val
                    if "25 percent" not in suggested_prcnt:
                        suggested_prcnt["25 percent"] = sugg_prcnt_val
                elif cum_prcnt >= 25:
                    suggested_prcnt["25 percent"] = sugg_prcnt_val
        else:
            result_dict = {}
            suggested_prcnt = {}
        game_dict[poll_name] = result_dict
        game_dict[f"{poll_name}_quartiles"] = suggested_prcnt
    return game_dict


def extract_poll_player_count(game_dict: dict, game_xml: bs4.element.Tag):
    player_count_poll = game_xml.find("poll", attrs={"name": "suggested_numplayers"})
    result_dict = {"total_votes": int(player_count_poll.attrs["totalvotes"])}
    player_count_results = player_count_poll.find_all("results")
    game_dict["player_count_recs"] = {}
    for player_count in player_count_results:
        num_players = player_count.attrs["numplayers"]
        player_count_values = {
            x.attrs["value"]: int(x.attrs["numvotes"])
            for x in player_count.find_all("result")
        }
        play_count_rec = max(player_count_values, key=player_count_values.get)
        if play_count_rec in game_dict["player_count_recs"]:
            game_dict["player_count_recs"][play_count_rec].append(num_players)
        else:
            game_dict["player_count_recs"][play_count_rec] = [num_players]
        result_dict[num_players] = player_count_values
        result_dict[num_players]["total_votes"] = sum(
            int(x.attrs["numvotes"]) for x in player_count.find_all("result")
        )
    game_dict["suggested_numplayers"] = result_dict
    return game_dict


def extract_version_info(game_dict: dict, game_xml: bs4.element.Tag):
    version_items = game_xml.find_all("item", attrs={"type": "boardgameversion"})
    version_list = []
    for vrs in version_items:
        version_dict = {
            "version_id": int(vrs["id"]),
            "version_nickname": vrs.find("name", attrs={"type": "primary"})["value"],
            "width": round(float(vrs.find("width")["value"])),
            "length": round(float(vrs.find("length")["value"])),
            "depth": round(float(vrs.find("depth")["value"])),
            "year_published": round(float(vrs.find("yearpublished")["value"])),
            "language": vrs.find("link", attrs={"type": "language"})["value"].lower(),
        }
        if vrs.find("thumbnail") is not None:
            version_dict["thumbnail"] = vrs.find("thumbnail").text
            version_dict["image"] = vrs.find("image").text
        if version_dict["width"] > 0:
            version_list.append(version_dict)
    if len(version_list) > 0:
        game_dict["versions"] = version_list
    return game_dict


def extract_ratings(game_dict: dict, game_xml: bs4.element.Tag):
    ratings_list = game_xml.find_all("comment")
    if "ratings" not in game_dict:
        game_dict["ratings"] = {}
    for rating in ratings_list:
        # round the rating to the nearest 0.5
        rating_round = round(2 * float(rating["rating"])) / 2
        if rating_round not in game_dict["ratings"]:
            game_dict["ratings"][rating_round] = [rating["username"]]
        else:
            game_dict["ratings"][rating_round].append(rating["username"])
    return game_dict


In [276]:
df_bg_list.loc[df_bg_list["id"].isin([2766, 2767, 2768])]

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
27559,2766,Jeopardy!,1964,27560,5.46425,5.26931,290,0,NaN,NaN,NaN,3306.0,NaN,NaN,NaN,NaN
28267,2767,Snail's Pace Race,1985,28268,5.38813,5.12660,770,0,NaN,NaN,981.0,NaN,NaN,NaN,NaN,NaN
28974,2768,Combo,1997,0,0.00000,4.88333,15,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
2767

In [240]:
df_bg_list.loc[df_bg_list["usersrated"]<500]

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
2767,388367,Innovation Ultimate,2025,2768,6.05302,8.65761,471,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3125,255249,Monikers: More Monikers,2018,3126,5.98430,8.28633,465,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3286,221248,Monikers: The Shut Up & Sit Down Nonsense Box,2017,3287,5.95818,8.04997,473,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3311,406174,Kinfire Delve: Callous' Lab,2024,3312,5.95416,8.21593,494,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3457,334363,Vijayanagara: The Deccan Empires of Medieval I...,2024,3458,5.93468,8.07951,463,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165457,447120,Pergola: Lavender Promo Tiles,2025,0,0.00000,0.00000,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165458,447122,"The Eyes of the Enemy: The Battle for Madrid, ...",2025,0,0.00000,0.00000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165459,447123,Marches & Batailles,2026,0,0.00000,0.00000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165460,447124,Tipni si!: Známe se? 18+,2020,0,0.00000,6.00000,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [277]:
bg_raw_data = get_boardgame_raw_data(df_bg_list.loc[df_bg_list["id"].isin([2766, 2767, 2768])])

2
3
4
5
6
7
8


In [278]:
df_bg_raw_data = pd.DataFrame(bg_raw_data)
df_bg_raw_data.head()

,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions,ratings
0,2766,https://cf.geekdo-images.com/IQuIkOvcveeoes1Sl...,https://cf.geekdo-images.com/IQuIkOvcveeoes1Sl...,3,5,45,45,45,12,"[Animals, Movies / TV / Radio theme, Party Gam...","[Betting and Bluffing, Open Drafting]","[Components: Sand Timers, TV Shows: Jeopardy!,...",[],[],[],[],[(Uncredited)],"[(Unknown), danspil, Gamestar, Igloo Books, Im...",[],1.62266,0.0,1.2353,1046,35,1,9,167,17,307,{},{},"{'total_votes': 2, '1': 0, '2': 0, '3': 0, '4'...",{'75 percent': 'Extensive use of text - massiv...,"{'Not Recommended': ['1', '2'], 'Recommended':...","{'total_votes': 3, '1': {'Best': 0, 'Recommend...","[{'version_id': 289079, 'version_nickname': 'd...","{1.0: ['Orgus', 'tobywan', 'Gosunkugi', 'DanZD..."
1,2767,https://cf.geekdo-images.com/tXlf1B32qWi1YeSaA...,https://cf.geekdo-images.com/tXlf1B32qWi1YeSaA...,2,6,10,10,10,3,"[Children's Game, Dice, Racing]",[Dice Rolling],"[Animals: Snails, Characters: Miffy]",[],"[Dick Bruna, Hans-Günther Döring, Horst Lauphe...",[],[],[Alex Randolph],"[(Unknown), Otto Maier Verlag, Ravensburger, (...",[],1.83479,0.0,1.0133,1489,46,18,92,396,75,771,"{'total_votes': 30, '2': 22, '3': 7, '4': 1, '...","{'50 percent': '2', '25 percent': '2', '75 per...","{'total_votes': 10, '6': 10, '7': 0, '8': 0, '...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '6+'], 'Recommended'...","{'total_votes': 20, '1': {'Best': 0, 'Recommen...","[{'version_id': 227821, 'version_nickname': 'D...","{1.0: ['SybotCB', 'Luke the Flaming', 'ZiggyZa..."
2,2768,https://cf.geekdo-images.com/5kyoy3HvpnfWQQY-w...,https://cf.geekdo-images.com/5kyoy3HvpnfWQQY-w...,2,4,10,10,10,8,[Dice],[Dice Rolling],[],[],[],[],[],[(Uncredited)],"[Jumbo, Pressman Toy Corp., Jumbo, Pressman To...",[],1.15060,0.0,1.0000,45,1,0,1,7,1,15,{},{},{},{},"{'Best': ['1', '2', '3', '4', '4+']}","{'total_votes': 0, '1': {'Best': 0, 'Recommend...","[{'version_id': 70516, 'version_nickname': 'Ju...","{2.0: ['arthemix'], 3.0: ['jdtaylo17'], 4.0: [..."


In [281]:
sum([len(v) for k,v in df_bg_raw_data["ratings"].iloc[2].items()])

15

# Code

## Pull ranks

In [5]:
df_tmp = pd.read_csv("../../data/processed/processed_games_language_dependence_1748995280.csv", sep="|", escapechar="\\")

In [7]:
df_tmp.dtypes

id                     int64
1                      int64
2                      int64
3                      int64
4                      int64
5                      int64
total_votes            int64
language_dependency    int64
dtype: object

In [3]:
cookies = crawler.get_driver_and_cookies()

In [29]:
df_boardgame_ranks = crawler.get_boardgame_ranks(cookies=cookies, save_file=True)

2025-05-26 10:36:33,973 - crawler.src.crawler - INFO - Fetching boardgame ranks
2025-05-26 10:36:34,810 - crawler.src.crawler - INFO - Successfully loaded 165498 boardgames


In [4]:
df_ranks = pd.read_csv("../../data/crawler/boardgame_ranks_20250526.csv")
df_ranks.head()

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank,queried_at_utc
0,224517,Brass: Birmingham,2018,1,8.40146,8.57715,52086,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2025-05-26T10:36:34
1,161936,Pandemic Legacy: Season 1,2015,2,8.36386,8.51619,55588,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,2025-05-26T10:36:34
2,342942,Ark Nova,2021,3,8.34620,8.53657,52459,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2025-05-26T10:36:34
3,174430,Gloomhaven,2017,4,8.32586,8.56341,64874,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,2025-05-26T10:36:34
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.22914,8.57682,26087,0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,2025-05-26T10:36:34


## pulling basic game info

In [116]:
# df_tmp = pd.read_csv(
#     "../../data/boardgame_data_raw_1748297111.csv",
#     sep="|",
#     escapechar="\\"
# )
# for col in [
#     "suggested_playerage",
#     "suggested_playerage_quartiles",
#     "language_dependence",
#     "language_dependence_quartiles",
#     "player_count_recs",
#     "suggested_numplayers",
#     "versions",
#     "ratings",
# ]:
#     df_tmp[col] = df_tmp[col].apply(lambda x: literal_eval(x))
# df_tmp.to_parquet("../../data/boardgame_data_raw_1748276896.parquet")    

In [23]:
df_ranks.columns

Index(['id', 'name', 'yearpublished', 'rank', 'bayesaverage', 'average',
       'usersrated', 'is_expansion', 'abstracts_rank', 'cgs_rank',
       'childrensgames_rank', 'familygames_rank', 'partygames_rank',
       'strategygames_rank', 'thematic_rank', 'wargames_rank',
       'queried_at_utc'],
      dtype='object')

In [7]:
(df_parq["description"].iloc[0])

"Brass: Birmingham is an economic strategy game sequel to Martin Wallace's 2007 masterpiece, Brass. Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution between the years of 1770 and 1870.\n\nIt offers a very different story arc and experience from its predecessor. As in its predecessor, you must develop, build and establish your industries and network in an effort to exploit low or high market demands. The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs. VPs are counted at the end of each half for the canals, rails and established (flipped) industry tiles.\n\nEach round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):\n\n1) Build - Pay required resources and place an industry tile.\n2) Network - Add a rail / canal link, expanding your network.\n3) 

In [9]:
df_parq = pd.read_parquet("../../data/crawler/boardgame_data_1749186734.parquet")
print(df_parq.shape)
df_parq.head()

(160, 37)


,id,thumbnail,image,description,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
0,224517,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,Brass: Birmingham is an economic strategy game...,2,4,120,60,120,14,"{'2726': 'Age of Reason', '1021': 'Economic', ...","{'2956': 'Chaining', '2875': 'End Game Bonuses...","{'17519': 'Cities: Birmingham (England)', '816...",{},"{'32887': 'Gavan Brown', '70571': 'Lina Cosset...",{},{'28720': 'Brass: Lancashire'},"{'32887': 'Gavan Brown', '32943': 'Matt Tolman...","{'21765': 'Roxley', '3475': 'Arclight Games', ...",{},1.42024,0.0,3.8686,74341,268,1748,19936,7193,2535,52546,"{'total_votes': 181, '2': 1, '3': 0, '4': 0, '...","{'25 percent': '12', '75 percent': '14', '50 p...","{'total_votes': 55, '1': 52, '2': 3, '3': 0, '...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '4+'], 'Recommended'...","{'total_votes': 1263, '1': {'Best': 0, 'Recomm...","[{'version_id': 523778, 'width': 12, 'length':..."
1,161936,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,Pandemic Legacy is a co-operative campaign gam...,2,4,60,60,60,13,"{'1084': 'Environmental', '2145': 'Medical'}","{'2001': 'Action Points', '2023': 'Cooperative...","{'64952': 'Components: Map (Global Scale)', '6...",{},{'14057': 'Chris Quilliams'},{},"{'221107': 'Pandemic Legacy: Season 2', '30549...","{'442': 'Rob Daviau', '378': 'Matt Leacock'}","{'538': 'Z-Man Games', '15889': 'Asterion Pres...",{},1.60226,0.0,2.8299,86819,498,794,14357,8322,1511,56125,"{'total_votes': 201, '2': 1, '3': 1, '4': 0, '...","{'25 percent': '10', '75 percent': '12', '50 p...","{'total_votes': 95, '6': 1, '7': 0, '8': 9, '9...",{'75 percent': 'Extensive use of text - massiv...,"{'Not Recommended': ['1', '4+'], 'Recommended'...","{'total_votes': 887, '1': {'Best': 21, 'Recomm...","[{'version_id': 329714, 'width': 9, 'length': ..."
2,342942,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,"In Ark Nova, you will plan and design a modern...",1,4,150,90,150,14,"{'1089': 'Animals', '1021': 'Economic', '1084'...","{'2689': 'Action Queue', '2875': 'End Game Bon...","{'110926': 'Animals: Okapi', '67874': 'Compone...","{'368966': 'Ark Nova: Marine Worlds', '433207'...","{'138547': 'Steffen Bieker', '11462': 'Loïc Bi...",{},{},{'138517': 'Mathias Wigge'},"{'22380': 'Feuerland Spiele', '30958': 'Capsto...",{},1.39214,0.0,3.7813,77119,393,1058,15670,6960,2753,52849,"{'total_votes': 292, '2': 0, '3': 0, '4': 0, '...","{'50 percent': '12', '25 percent': '12', '75 p...","{'total_votes': 73, '11': 0, '12': 0, '13': 25...",{'25 percent': 'Moderate in-game text - needs ...,"{'Recommended': ['1', '3'], 'Best': ['2'], 'No...","{'total_votes': 2093, '1': {'Best': 207, 'Reco...","[{'version_id': 623699, 'width': 12, 'length':..."
3,174430,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,Gloomhaven is a game of Euro-inspired tactica...,1,4,120,60,120,14,"{'1022': 'Adventure', '1020': 'Exploration', '...","{'2689': 'Action Queue', '2839': 'Action Retri...","{'59218': 'Category: Dungeon Crawler', '67953'...","{'365186': 'The Crimson Scales', '367749': 'Th...","{'77084': 'Alexandr Elichev', '78961': 'Josh T...",{},{'390478': 'Gloomhaven (Second Edition)'},{'69802': 'Isaac Childres'},"{'27425': 'Cephalofair Games', '4304': 'Albi',...","{'295770': 'Frosthaven', '291457': 'Gloomhaven...",1.73741,0.0,3.9142,101233,1184,11

In [10]:
df_parq["description"].iloc[0]

"Brass: Birmingham is an economic strategy game sequel to Martin Wallace's 2007 masterpiece, Brass. Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution between the years of 1770 and 1870.&#10;&#10;It offers a very different story arc and experience from its predecessor. As in its predecessor, you must develop, build and establish your industries and network in an effort to exploit low or high market demands. The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs. VPs are counted at the end of each half for the canals, rails and established (flipped) industry tiles.&#10;&#10;Each round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):&#10;&#10;1) Build - Pay required resources and place an industry tile.&#10;2) Network - Add a rail / canal link, expandi

In [11]:
df_parq = pd.read_parquet("../../data/crawler/boardgame_simple_data_1749187980.parquet")
print(df_parq.shape)
df_parq.head()

(400, 30)


,id,thumbnail,image,description,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings
0,224517,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,Brass: Birmingham is an economic strategy game...,2,4,120,60,120,14,"{'2726': 'Age of Reason', '1021': 'Economic', ...","{'2956': 'Chaining', '2875': 'End Game Bonuses...","{'17519': 'Cities: Birmingham (England)', '816...",{},"{'32887': 'Gavan Brown', '70571': 'Lina Cosset...",{},{'28720': 'Brass: Lancashire'},"{'32887': 'Gavan Brown', '32943': 'Matt Tolman...","{'21765': 'Roxley', '3475': 'Arclight Games', ...",{},1.42024,0.0,3.8686,74341,268,1748,19936,7193,2535,52546
1,161936,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,Pandemic Legacy is a co-operative campaign gam...,2,4,60,60,60,13,"{'1084': 'Environmental', '2145': 'Medical'}","{'2001': 'Action Points', '2023': 'Cooperative...","{'64952': 'Components: Map (Global Scale)', '6...",{},{'14057': 'Chris Quilliams'},{},"{'221107': 'Pandemic Legacy: Season 2', '30549...","{'442': 'Rob Daviau', '378': 'Matt Leacock'}","{'538': 'Z-Man Games', '15889': 'Asterion Pres...",{},1.60226,0.0,2.8299,86819,498,794,14357,8322,1511,56125
2,342942,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,"In Ark Nova, you will plan and design a modern...",1,4,150,90,150,14,"{'1089': 'Animals', '1021': 'Economic', '1084'...","{'2689': 'Action Queue', '2875': 'End Game Bon...","{'110926': 'Animals: Okapi', '67874': 'Compone...","{'368966': 'Ark Nova: Marine Worlds', '433207'...","{'138547': 'Steffen Bieker', '11462': 'Loïc Bi...",{},{},{'138517': 'Mathias Wigge'},"{'22380': 'Feuerland Spiele', '30958': 'Capsto...",{},1.39214,0.0,3.7813,77119,393,1058,15670,6960,2753,52849
3,174430,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,Gloomhaven is a game of Euro-inspired tactica...,1,4,120,60,120,14,"{'1022': 'Adventure', '1020': 'Exploration', '...","{'2689': 'Action Queue', '2839': 'Action Retri...","{'59218': 'Category: Dungeon Crawler', '67953'...","{'365186': 'The Crimson Scales', '367749': 'Th...","{'77084': 'Alexandr Elichev', '78961': 'Josh T...",{},{'390478': 'Gloomhaven (Second Edition)'},{'69802': 'Isaac Childres'},"{'27425': 'Cephalofair Games', '4304': 'Albi',...","{'295770': 'Frosthaven', '291457': 'Gloomhaven...",1.73741,0.0,3.9142,101233,1184,1165,21423,11305,2670,65301
4,233078,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,Twilight Imperium (Fourth Edition) is a game o...,3,6,480,240,480,14,"{'1015': 'Civilization', '1021': 'Economic', '...","{'2838': 'Action Drafting', '2021': 'Area-Impu...","{'67874': 'Components: Hexagonal Tiles', '6494...",{'315895': 'Twilight Imperium: Fourth Edition ...,{'11988': 'Scott Schomburg'},{},{'12493': 'Twilight Imperium: Third Edition'},"{'96049': 'Dane Beltrami', '6651': 'Corey Koni...","{'17': 'Fantasy Flight Games', '23043': 'ADC B...",{},1.63981,0.0,4.3349,31480,211,1060,12258,3902,1278,26372


In [159]:
combined_dict = {}
for d in df_parq["boardgamecategory"]:
    combined_dict.update(d)

In [182]:
boardgamecategory_rows = []
col = "boardgamecategory"
for _, row in df_parq.iterrows():
    if isinstance(row[col], dict):  # Ensure it's a dict
        for k, v in row[col].items():
            boardgamecategory_rows.append({"game_id": row["id"], f"{col}_id": k, f"{col}_name": v})
df_category_row = pd.DataFrame(boardgamecategory_rows)

In [194]:
df_merged.head()

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
0,224517,Brass: Birmingham,2018,1,8.40146,8.57715,52086,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,2,4,120,60,120,14,"{'2726': 'Age of Reason', '1021': 'Economic', ...","{'2956': 'Chaining', '2040': 'Hand Management'...","{'17519': 'Cities: Birmingham (England)', '816...",{},"{'32887': 'Gavan Brown', '70571': 'Lina Cosset...",{},{'28720': 'Brass: Lancashire'},"{'32887': 'Gavan Brown', '32943': 'Matt Tolman...","{'21765': 'Roxley', '3475': 'Arclight Games', ...",{},1.41952,0.0,3.8680,74212,265,1757,19928,7182,2530,52464,"{'total_votes': 181, '2': 1, '3': 0, '4': 0, '...","{'25 percent': '12', '75 percent': '14', '50 p...","{'total_votes': 55, '11': 52, '12': 3, '13': 0...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '4+'], 'Recommended'...","{'total_votes': 1258, '1': {'Best': 0, 'Recomm...","[{'version_id': 523778, 'width': 12, 'length':..."
1,161936,Pandemic Legacy: Season 1,2015,2,8.36386,8.51619,55588,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,2,4,60,60,60,13,"{'1084': 'Environmental', '2145': 'Medical'}","{'2001': 'Action Points', '2023': 'Cooperative...","{'64952': 'Components: Map (Global Scale)', '6...",{},{'14057': 'Chris Quilliams'},{},"{'221107': 'Pandemic Legacy: Season 2', '30549...","{'442': 'Rob Daviau', '378': 'Matt Leacock'}","{'538': 'Z-Man Games', '15889': 'Asterion Pres...",{},1.60115,0.0,2.8299,86772,499,794,14353,8319,1511,56094,"{'total_votes': 201, '2': 1, '3': 1, '4': 0, '...","{'25 percent': '10', '75 percent': '12', '50 p...","{'total_votes': 95, '1': 1, '2': 0, '3': 9, '4...",{'75 percent': 'Extensive use of text - massiv...,"{'Not Recommended': ['1', '4+'], 'Recommended'...","{'total_votes': 887, '1': {'Best': 21, 'Recomm...","[{'version_id': 329714, 'width': 9, 'length': ..."
2,342942,Ark Nova,2021,3,8.34620,8.53657,52459,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,1,4,150,90,150,14,"{'1089': 'Animals', '1021': 'Economic', '1084'...","{'2689': 'Action Queue', '2875': 'End Game Bon...","{'110926': 'Animals: Okapi', '67874': 'Compone...","{'368966': 'Ark Nova: Marine Worlds', '433207'...","{'138547': 'Steffen Bieker', '11462': 'Loïc Bi...",{},{},{'138517': 'Mathias Wigge'},"{'22380': 'Feuerland Spiele', '30958': 'Capsto...",{},1.39303,0.0,3.7812,76971,391,1063,15649,6948,2751,52749,"{'total_votes': 291, '2': 0, '3': 0, '4': 0, '...","{'50 percent': '12', '25 percent': '12', '75 p...","{'total_votes': 73, '6': 0, '7': 0, '8': 25, '...",{'25 percent': 'Moderate in-game text - needs ...,"{'Recommended': ['1', '3'], 'Best': ['2'], 'No...","{'total_votes': 2090, '1': {'Best': 207, 'Reco...","[{'version_id': 623699, 'width': 12, 'length':..."
3,174430,Gloomhaven,2017,4,8.32586,8.56341,64874,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,1,4,120,60,120,14,"{'1022': 'Adventure', '1020': 'Exploration', '...","{'2689': 'Action Queue', '2839': 'Action Retri...","{'59218': 'Category: Dungeon Crawler', '67953'...","{'365186': 'The C

In [210]:
df_full = pd.read_parquet("../../data/crawler/boardgame_data_1748875957.parquet")

In [207]:
poss_cat = [
    "boardgamecategory",
    "boardgamemechanic",
    "boardgamefamily",
    "boardgameexpansion",
    "boardgameartist",
    "boardgamecompilation",
    "boardgameimplementation",
    "boardgamedesigner",
    "boardgamepublisher",
    "boardgameintegration",
]
col = "boardgameexpansion"
df_exploded = pd.DataFrame(
    [
        {"game_id": row_id, f"{col}_id": k, f"{col}_name": v}
        for row_id, d in zip(df_parq["id"], df_parq[col])
        if isinstance(d, dict)  # ensure it's a dict
        for k, v in d.items()
    ]
)
df_exploded[f"{col}_id"] = df_exploded[f"{col}_id"].astype(int)
df_exploded.head()

,game_id,boardgameexpansion_id,boardgameexpansion_name
0,342942,368966,Ark Nova: Marine Worlds
1,342942,433207,Ark Nova: Promotion Team & Capybara Promo Cards
2,342942,368158,Ark Nova: Zoo Map Pack 1
3,342942,426978,Ark Nova: Zoo Map Pack 2
4,174430,365186,The Crimson Scales


In [291]:
df_parq["language_dependence"].iloc[3]

{'total_votes': 70, '11': 1, '12': 0, '13': 2, '14': 47, '15': 20}

In [292]:
df_lang = df_full.loc[~(df_full["language_dependence"]=={}), ["id", "language_dependence"]]
df_lang["game_total_votes"] = df_lang["language_dependence"].apply(lambda x: x["total_votes"])
df_lang = df_lang.loc[df_lang["game_total_votes"]>10]
df_lang

,id,language_dependence,game_total_votes
0,224517,"{'total_votes': 55, '11': 52, '12': 3, '13': 0...",55
1,161936,"{'total_votes': 95, '1': 1, '2': 0, '3': 9, '4...",95
2,342942,"{'total_votes': 73, '6': 0, '7': 0, '8': 25, '...",73
3,174430,"{'total_votes': 70, '11': 1, '12': 0, '13': 2,...",70
4,233078,"{'total_votes': 28, '16': 0, '17': 0, '18': 2,...",28
...,...,...,...
162130,258210,"{'total_votes': 16, '31': 10, '32': 3, '33': 3...",16
162483,258779,"{'total_votes': 20, '1': 0, '2': 2, '3': 18, '...",20
162938,391163,"{'total_votes': 21, '76': 0, '77': 1, '78': 15...",21
162986,260180,"{'total_votes': 19, '11': 19, '12': 0, '13': 0...",19


In [323]:
df_lang = df_full.loc[~(df_full["language_dependence"]=={}), ["id", "language_dependence"]]
df_lang["game_total_votes"] = df_lang["language_dependence"].apply(lambda x: x["total_votes"])
df_lang = df_lang.loc[df_lang["game_total_votes"]>10]
col = "language_dependence"
df_exploded = pd.DataFrame(
    [
        {"id": row_id, col: k, "votes": v}
        for row_id, d in zip(df_lang["id"], df_lang[col])
        if isinstance(d, dict)  # ensure it's a dict
        for k, v in d.items()
    ]
)
df_exploded = df_exploded.loc[(df_exploded[col] != "total_votes")]
df_exploded["language_dependence"] = df_exploded["language_dependence"].astype(int)
df_min = (
    df_exploded[["id", "language_dependence"]]
    .groupby("id")
    .min()
    .reset_index()
    .rename(columns={"language_dependence": "lang_min"})
)
df_exploded = df_exploded.merge(df_min, how="left", on="id")
df_exploded["language_dependence_norm"] = (
    df_exploded["language_dependence"] - df_exploded["lang_min"] + 1
)
df_exploded = df_exploded.pivot(index="id", columns="language_dependence_norm", values="votes")
df_exploded["total_votes"] = df_exploded.sum(axis=1)
df_exploded["language_dependency"] = df_exploded[[1,2,3,4,5]].idxmax(axis=1)
df_exploded = df_exploded.reset_index()
df_exploded.columns.name=None
# df_exploded = pd.concat(
#     [df_lang.drop(columns="votes"), pd.json_normalize(df_exploded["votes"])], axis=1
# )
# df_exploded = df_exploded.loc[
#     (df_exploded[col] != "total_votes") & ~(df_exploded[col].str.contains("+", regex=False))
# ]
# df_exploded = df_exploded.merge(df_sugg_players[["id", "game_total_votes"]], on="id", how="left")
# df_exploded.columns = [x.lower().replace(" ", "_") for x in df_exploded.columns]
# for col in df_exploded.columns:
#     df_exploded[col] = df_exploded[col].astype(pd.Int64Dtype())
# df_exploded["recommendation"] = df_exploded[["best", "recommended", "not_recommended"]].idxmax(
#     axis=1
# )
# df_exploded = df_exploded.rename(columns={"suggested_numplayers": "players", "id": "game_id"})
df_exploded.head(20)

,id,1,2,3,4,5,total_votes,language_dependency
0,1,37,5,7,0,0,49,1
1,3,76,0,1,0,0,77,1
2,5,76,13,1,0,0,90,1
3,10,34,0,0,0,0,34,1
4,11,141,8,0,0,0,149,1
5,12,83,7,0,0,0,90,1
6,13,21,294,30,1,0,346,2
7,15,0,1,8,5,0,14,3
8,18,7,22,50,1,0,80,3
9,22,0,2,8,3,2,15,3


In [322]:
df_exploded.columns

Index(['id', 1, 2, 3, 4, 5, 'total_votes'], dtype='object')

In [313]:
df_exploded["language_dependence_norm"].value_counts()

language_dependence_norm
1    2043
2    2043
3    2043
4    2043
5    2043
Name: count, dtype: int64

In [295]:
col = "language_dependence"
# df_exploded = pd.DataFrame(
#     [
#         {"id": row_id, col: k, "votes": v}
#         for row_id, d in zip(df_lang["id"], df_lang[col])
#         if isinstance(d, dict)  # ensure it's a dict
#         for k, v in d.items()
#     ]
# )
df_exploded = pd.concat(
    [df_lang.drop(columns=col), pd.json_normalize(df_lang[col])], axis=1
)
# df_exploded = df_exploded.loc[
#     (df_exploded[col] != "total_votes") & ~(df_exploded[col].str.contains("+", regex=False))
# ]
# df_exploded = df_exploded.merge(df_sugg_players[["id", "game_total_votes"]], on="id", how="left")
# df_exploded.columns = [x.lower().replace(" ", "_") for x in df_exploded.columns]
# for col in df_exploded.columns:
#     df_exploded[col] = df_exploded[col].astype(pd.Int64Dtype())
# df_exploded["recommendation"] = df_exploded[["best", "recommended", "not_recommended"]].idxmax(
#     axis=1
# )
# df_exploded = df_exploded.rename(columns={"suggested_numplayers": "players", "id": "game_id"})
df_exploded.head()

,id,game_total_votes,total_votes,11,12,13,14,15,1,2,3,4,5,6,7,8,9,10,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,224517.0,55.0,55.0,52.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,161936.0,95.0,95.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,9.0,64.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,342942.0,73.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,25.0,44.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,174430.0,70.0,70.0,1.0,0.0,2.0,47.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,233078.0,28.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0,20.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
col = "versions"
df_exploded = pd.DataFrame(
    [
        {"id": row_id, col: k, "votes": v}
        for row_id, d in zip(df_sugg_players["id"], df_sugg_players[col])
        if isinstance(d, dict)  # ensure it's a dict
        for k, v in d.items()
    ]
)
df_exploded = pd.concat(
    [df_exploded.drop(columns="votes"), pd.json_normalize(df_exploded["votes"])], axis=1
)

In [281]:
df_versions = df_full.loc[df_full["versions"].notna(),["id","versions"]].explode("versions").reset_index(drop=True)
df_versions = pd.concat(
    [df_versions.drop(columns="versions"), pd.json_normalize(df_versions["versions"])], axis=1
)
df_versions = df_versions.rename(columns={"id":"game_id"})
df_versions.head()

,game_id,version_id,width,length,depth,year_published,thumbnail,image,language,version_nickname
0,224517,523778,12,12,3,2020,https://cf.geekdo-images.com/9R-zfmWa3EjsPeFNW...,https://cf.geekdo-images.com/9R-zfmWa3EjsPeFNW...,chinese,Chinese edition
1,224517,712133,12,12,2,2024,https://cf.geekdo-images.com/9d9YmbOjTy7-rjwWM...,https://cf.geekdo-images.com/9d9YmbOjTy7-rjwWM...,czech,Czech edition 2024
2,224517,745587,12,12,2,2025,https://cf.geekdo-images.com/9d9YmbOjTy7-rjwWM...,https://cf.geekdo-images.com/9d9YmbOjTy7-rjwWM...,czech,Czech edition 2025
3,224517,558467,12,12,2,2020,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,czech,"Czech/English edition, first printing"
4,224517,712131,12,12,2,2021,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,czech,"Czech/English edition, second printing"


In [282]:
df_versions.dtypes

game_id              int64
version_id           int64
width                int64
length               int64
depth                int64
year_published       int64
thumbnail           object
image               object
language            object
version_nickname    object
dtype: object

In [278]:
pd.json_normalize(df_versions["versions"])

,version_id,width,length,depth,year_published,thumbnail,image,language,version_nickname
0,523778,12,12,3,2020,https://cf.geekdo-images.com/9R-zfmWa3EjsPeFNW...,https://cf.geekdo-images.com/9R-zfmWa3EjsPeFNW...,chinese,Chinese edition
1,712133,12,12,2,2024,https://cf.geekdo-images.com/9d9YmbOjTy7-rjwWM...,https://cf.geekdo-images.com/9d9YmbOjTy7-rjwWM...,czech,Czech edition 2024
2,745587,12,12,2,2025,https://cf.geekdo-images.com/9d9YmbOjTy7-rjwWM...,https://cf.geekdo-images.com/9d9YmbOjTy7-rjwWM...,czech,Czech edition 2025
3,558467,12,12,2,2020,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,czech,"Czech/English edition, first printing"
4,712131,12,12,2,2021,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,czech,"Czech/English edition, second printing"
...,...,...,...,...,...,...,...,...,...
156526,424325,12,12,3,2019,https://cf.geekdo-images.com/pH5yaZoOqqSeWAX3S...,https://cf.geekdo-images.com/pH5yaZoOqqSeWAX3S...,english,English edition
156527,669008,4,5,1,2015,NaN,NaN,japanese,Japanese edition
156528,424330,24,34,0,0,NaN,NaN,spanish,Spanish edition
156529,424332,4,5,1,2019,https://cf.geekdo-images.com/GUEbkRMulkEdIRLT_...,https://cf.geekdo-images.com/GUEbkRMulkEdIRLT_...,english,English edition


In [224]:
df_sugg_players["suggested_numplayers"].iloc[-1]

{'total_votes': 37,
 '1': {'Best': 0, 'Recommended': 0, 'Not Recommended': 23, 'total_votes': 23},
 '2': {'Best': 0, 'Recommended': 6, 'Not Recommended': 19, 'total_votes': 25},
 '3': {'Best': 3, 'Recommended': 16, 'Not Recommended': 7, 'total_votes': 26},
 '4': {'Best': 17, 'Recommended': 13, 'Not Recommended': 1, 'total_votes': 31},
 '5': {'Best': 21, 'Recommended': 10, 'Not Recommended': 0, 'total_votes': 31},
 '6': {'Best': 12, 'Recommended': 15, 'Not Recommended': 2, 'total_votes': 29},
 '6+': {'Best': 8, 'Recommended': 9, 'Not Recommended': 7, 'total_votes': 24}}

In [222]:
df_sugg_players.loc[df_sugg_players["total_votes"]>10].shape

(7234, 3)

In [217]:
df_full.loc[~(df_full["suggested_numplayers"]=={})]

,id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
0,224517,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,2,4,120,60,120,14,"{'2726': 'Age of Reason', '1021': 'Economic', ...","{'2956': 'Chaining', '2040': 'Hand Management'...","{'17519': 'Cities: Birmingham (England)', '816...",{},"{'32887': 'Gavan Brown', '70571': 'Lina Cosset...",{},{'28720': 'Brass: Lancashire'},"{'32887': 'Gavan Brown', '32943': 'Matt Tolman...","{'21765': 'Roxley', '3475': 'Arclight Games', ...",{},1.419520,0.0,3.8680,74212,265,1757,19928,7182,2530,52464,"{'total_votes': 181, '2': 1, '3': 0, '4': 0, '...","{'25 percent': '12', '75 percent': '14', '50 p...","{'total_votes': 55, '11': 52, '12': 3, '13': 0...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '4+'], 'Recommended'...","{'total_votes': 1258, '1': {'Best': 0, 'Recomm...","[{'version_id': 523778, 'width': 12, 'length':..."
1,161936,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,2,4,60,60,60,13,"{'1084': 'Environmental', '2145': 'Medical'}","{'2001': 'Action Points', '2023': 'Cooperative...","{'64952': 'Components: Map (Global Scale)', '6...",{},{'14057': 'Chris Quilliams'},{},"{'221107': 'Pandemic Legacy: Season 2', '30549...","{'442': 'Rob Daviau', '378': 'Matt Leacock'}","{'538': 'Z-Man Games', '15889': 'Asterion Pres...",{},1.601150,0.0,2.8299,86772,499,794,14353,8319,1511,56094,"{'total_votes': 201, '2': 1, '3': 1, '4': 0, '...","{'25 percent': '10', '75 percent': '12', '50 p...","{'total_votes': 95, '1': 1, '2': 0, '3': 9, '4...",{'75 percent': 'Extensive use of text - massiv...,"{'Not Recommended': ['1', '4+'], 'Recommended'...","{'total_votes': 887, '1': {'Best': 21, 'Recomm...","[{'version_id': 329714, 'width': 9, 'length': ..."
2,342942,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,1,4,150,90,150,14,"{'1089': 'Animals', '1021': 'Economic', '1084'...","{'2689': 'Action Queue', '2875': 'End Game Bon...","{'110926': 'Animals: Okapi', '67874': 'Compone...","{'368966': 'Ark Nova: Marine Worlds', '433207'...","{'138547': 'Steffen Bieker', '11462': 'Loïc Bi...",{},{},{'138517': 'Mathias Wigge'},"{'22380': 'Feuerland Spiele', '30958': 'Capsto...",{},1.393030,0.0,3.7812,76971,391,1063,15649,6948,2751,52749,"{'total_votes': 291, '2': 0, '3': 0, '4': 0, '...","{'50 percent': '12', '25 percent': '12', '75 p...","{'total_votes': 73, '6': 0, '7': 0, '8': 25, '...",{'25 percent': 'Moderate in-game text - needs ...,"{'Recommended': ['1', '3'], 'Best': ['2'], 'No...","{'total_votes': 2090, '1': {'Best': 207, 'Reco...","[{'version_id': 623699, 'width': 12, 'length':..."
3,174430,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,1,4,120,60,120,14,"{'1022': 'Adventure', '1020': 'Exploration', '...","{'2689': 'Action Queue', '2839': 'Action Retri...","{'59218': 'Category: Dungeon Crawler', '67953'...","{'365186': 'The Crimson Scales', '367749': 'Th...","{'77084': 'Alexandr Elichev', '78961': 'Josh T...",{},{'390478': 'Gloomhaven (Second Edition)'},{'69802': 'Isaac Childres'},"{'27425': 'Cephalofair Games', '4304': 'Albi',...","{'295770': 'Frosthaven', '291457': 'Gloomhaven...",1.737110,0.0,3.9142,101172,1182,1165,21423,11310,2669,65274,"{'total_votes': 330, '2': 5, '3': 0, '4': 0, '...","{'25 percent': '12', '75 percent': '14', '50 p...","{'total_votes': 70, '11': 1, '12': 0, '13': 2,...",{'50 percent': 'Extensive u

In [212]:
df_full["suggested_numplayers"].apply(lambda x: x.get("total_votes"))

0         1258.0
1          887.0
2         2090.0
3         1592.0
4          621.0
           ...  
164100       NaN
164101       NaN
164102       1.0
164103       3.0
164104       1.0
Name: suggested_numplayers, Length: 164105, dtype: float64

In [208]:
df_exploded.dtypes

game_id                     int64
boardgameexpansion_id       int64
boardgameexpansion_name    object
dtype: object

In [200]:
df_merged["player_count_recs"].iloc[4]

{'Not Recommended': ['1', '2', '6+'],
 'Recommended': ['3', '4', '5'],
 'Best': ['6']}

In [199]:
df_merged["suggested_numplayers"].iloc[4]

{'total_votes': 621,
 '1': {'Best': 0,
  'Recommended': 5,
  'Not Recommended': 437,
  'total_votes': 442},
 '2': {'Best': 0,
  'Recommended': 19,
  'Not Recommended': 426,
  'total_votes': 445},
 '3': {'Best': 22,
  'Recommended': 248,
  'Not Recommended': 212,
  'total_votes': 482},
 '4': {'Best': 169,
  'Recommended': 294,
  'Not Recommended': 73,
  'total_votes': 536},
 '5': {'Best': 117,
  'Recommended': 357,
  'Not Recommended': 48,
  'total_votes': 522},
 '6': {'Best': 445,
  'Recommended': 91,
  'Not Recommended': 28,
  'total_votes': 564},
 '6+': {'Best': 12,
  'Recommended': 67,
  'Not Recommended': 259,
  'total_votes': 338}}

In [197]:
del(df_category_row)

In [177]:
df_tmp = df_parq[["id", "boardgamecategory"]].copy()
# Step 1: Convert dictionary into Series (key-value pairs as rows)
df_exploded = df_tmp.explode(df_tmp["boardgamecategory"].apply(lambda d: list(d.items())))

# Step 2: Extract keys and values from the tuples
df_exploded[["key", "value"]] = pd.DataFrame(
    df_exploded["boardgamecategory"].tolist(), index=df_exploded.index
)

# df_tmp["boardgamecategory"] = df_tmp["boardgamecategory"].apply(lambda x: list(x.keys()))
# # Step 1: Explode the list column
# exploded = df_tmp.explode('boardgamecategory')

# # Step 2: Create one-hot encoding
# one_hot = pd.get_dummies(exploded['boardgamecategory'])

# # Step 3: Group back to original rows (sum by original index)
# one_hot_encoded = one_hot.groupby(exploded.index).sum()

# # Step 4: Combine with original DataFrame
# result = df_tmp.drop(columns='boardgamecategory').join(one_hot_encoded)

ValueError: column must be a scalar, tuple, or list thereof

In [174]:
df_tmp

,id,boardgamecategory
0,224517,"{'2726': 'Age of Reason', '1021': 'Economic', ..."
1,161936,"{'1084': 'Environmental', '2145': 'Medical'}"
2,342942,"{'1089': 'Animals', '1021': 'Economic', '1084'..."
3,174430,"{'1022': 'Adventure', '1020': 'Exploration', '..."
4,233078,"{'1015': 'Civilization', '1021': 'Economic', '..."
...,...,...
95510,245931,{'1015': 'Civilization'}
95511,245932,"{'1089': 'Animals', '1021': 'Economic', '1042'..."
95512,245934,{'1050': 'Ancient'}
95513,245941,"{'1017': 'Dice', '1064': 'Movies / TV / Radio ..."


In [175]:
df_tmp.to_dict(orient="records")

[{'id': 224517,
  'boardgamecategory': {'2726': 'Age of Reason',
   '1021': 'Economic',
   '1088': 'Industry / Manufacturing',
   '2710': 'Post-Napoleonic',
   '1034': 'Trains',
   '1011': 'Transportation'}},
 {'id': 161936,
  'boardgamecategory': {'1084': 'Environmental', '2145': 'Medical'}},
 {'id': 342942,
  'boardgamecategory': {'1089': 'Animals',
   '1021': 'Economic',
   '1084': 'Environmental'}},
 {'id': 174430,
  'boardgamecategory': {'1022': 'Adventure',
   '1020': 'Exploration',
   '1010': 'Fantasy',
   '1046': 'Fighting',
   '1047': 'Miniatures'}},
 {'id': 233078,
  'boardgamecategory': {'1015': 'Civilization',
   '1021': 'Economic',
   '1020': 'Exploration',
   '1026': 'Negotiation',
   '1001': 'Political',
   '1016': 'Science Fiction',
   '1113': 'Space Exploration',
   '1086': 'Territory Building',
   '1019': 'Wargame'}},
 {'id': 316554,
  'boardgamecategory': {'1064': 'Movies / TV / Radio theme',
   '1093': 'Novel-based',
   '1001': 'Political',
   '1016': 'Science Ficti

In [163]:
len(combined_dict.keys())

84

In [128]:
pd.Series(df_tmp['numratings'].values, index=df_tmp['game_id']).to_dict()

{13: 135581,
 822: 134531,
 30549: 131224,
 68448: 109293,
 167791: 106995,
 266192: 104207,
 173346: 102440,
 230802: 100285,
 178900: 98721,
 9209: 94713,
 36218: 94378,
 169786: 89213,
 148228: 85728,
 14996: 81377,
 31260: 75232,
 70323: 73775,
 40692: 73607,
 3076: 73094,
 163412: 71053,
 129622: 71259}

In [130]:
int(df_tmp["ratings_pulled"].min()/100)

560

In [6]:
import importlib
importlib.reload(crawler)

<module 'crawler.src.crawler' from '/home/ubuntu/git/pax_tt_recommender/crawler/src/crawler.py'>

In [75]:
import requests
from bs4 import BeautifulSoup
batch_ids = ['245952']
bg_info_url = f"https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame,boardgameexpansion&stats=1&versions=1&ratingcomments=1&pagesize=100&page=1&id={','.join(batch_ids)}"
print(bg_info_url)
# bgg_response = requests.get(bg_info_url)
# soup_xml = BeautifulSoup(bgg_response.content, "xml")
# games_xml_list = soup_xml.find_all("item", attrs={"type": ["boardgame", "boardgameexpansion"]})
# len(games_xml_list)

https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame,boardgameexpansion&stats=1&versions=1&ratingcomments=1&pagesize=100&page=1&id=245952


In [13]:
# batch_ids = [str(x) for x in df_ranks.iloc[105293:105313]["id"].tolist()]
batch_ids = '13'
bg_info_url = f"https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame,boardgameexpansion&stats=1&versions=1&ratingcomments=1&pagesize=100&page=1&id={','.join(batch_ids)}"
bgg_response = requests.get(bg_info_url)
soup_xml = BeautifulSoup(bgg_response.content, "xml")
games_xml_list = soup_xml.find_all("item", attrs={"type": ["boardgame", "boardgameexpansion"]})

In [19]:
import html
print(html.unescape(games_xml_list[0].find("description").text))

Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring victory points. First, each regional election can supply one to eighty victory points, depending on the size of the region and how well your party does in it. Second, if a party wins a regional election and has some media influence in the region, then the party will receive some media-control victory points. Third, each party has a national party membership which will grow as the game progresses and this will supply a fair number of victory points. Lastly, parties score some victory points if their party platform matches the national opinions at the end of the game.

The 1986 edition featured four parties from the old West Germany and supported 3-4 play

In [14]:
games_xml_list

[<item id="1" type="boardgame">
 <thumbnail>https://cf.geekdo-images.com/rpwCZAjYLD940NWwP3SRoA__thumb/img/YT6svCVsWqLrDitcMEtyazVktbQ=/fit-in/200x150/filters:strip_icc()/pic4718279.jpg</thumbnail>
 <image>https://cf.geekdo-images.com/rpwCZAjYLD940NWwP3SRoA__original/img/yR0aoBVKNrAmmCuBeSzQnMflLYg=/0x0/filters:format(jpeg)/pic4718279.jpg</image>
 <name sortindex="5" type="primary" value="Die Macher"/>
 <name sortindex="1" type="alternate" value="德国大选"/>
 <name sortindex="1" type="alternate" value="디 마허"/>
 <description>Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring victory points. First, each regional election can supply one to eighty victory points, depending on the size of the region and how well

## Merge data

In [151]:
df_merged = df_ranks.head().drop(columns=['queried_at_utc']).merge(df_parq.head(), on="id", how="inner")
games_data = df_merged.to_dict(orient="records")

In [118]:
df_parq["minplayers"].value_counts(dropna=False)

minplayers
2     70136
1     12803
3      7418
0      2205
4      2155
5       224
6       208
8       198
7        58
10       41
9        21
12       15
15        7
20        5
11        4
13        4
16        4
14        3
40        2
24        1
25        1
18        1
50        1
Name: count, dtype: int64

In [128]:
import numpy as np

In [147]:
df_test = pd.DataFrame({"col": [1.23, 2, 3, np.nan, None]})
df_test["col"] = df_test["col"].fillna(value=pd.NA).astype(pd.Float64Dtype())
df_test

,col
0,1.23
1,2.0
2,3.0
3,<NA>
4,<NA>


In [148]:
df_test.to_dict(orient="records")

[{'col': 1.23}, {'col': 2.0}, {'col': 3.0}, {'col': None}, {'col': None}]

In [12]:
df_parq = pd.read_parquet("../../data/crawler/boardgame_data_1748875957.parquet")
df_parq.shape

(164105, 36)

In [17]:
df_parq_simple = pd.read_parquet("../../data/crawler/boardgame_simple_data_1749221749.parquet", columns=["id", "description"])
df_parq_simple.shape

(164208, 2)

In [19]:
df_parq_merged = df_parq.merge(df_parq_simple, on="id", how="left")
df_parq_merged.shape

(164105, 37)

In [25]:
from time import sleep, time
int(time())

1749253240

In [26]:
df_parq_merged.columns

Index(['id', 'thumbnail', 'image', 'minplayers', 'maxplayers', 'playingtime',
       'minplaytime', 'maxplaytime', 'minage', 'boardgamecategory',
       'boardgamemechanic', 'boardgamefamily', 'boardgameexpansion',
       'boardgameartist', 'boardgamecompilation', 'boardgameimplementation',
       'boardgamedesigner', 'boardgamepublisher', 'boardgameintegration',
       'stddev', 'median', 'averageweight', 'owned', 'trading', 'wanting',
       'wishing', 'numcomments', 'numweights', 'numratings',
       'suggested_playerage', 'suggested_playerage_quartiles',
       'language_dependence', 'language_dependence_quartiles',
       'player_count_recs', 'suggested_numplayers', 'versions', 'description'],
      dtype='object')

In [ ]:
[
    "id",
    "thumbnail",
    "image",
    "minplayers",
    "maxplayers",
    "playingtime",
    "minplaytime",
    "maxplaytime",
    "minage",
    "boardgamecategory",
    "boardgamemechanic",
    "boardgamefamily",
    "boardgameexpansion",
    "boardgameartist",
    "boardgamecompilation",
    "boardgameimplementation",
    "boardgamedesigner",
    "boardgamepublisher",
    "boardgameintegration",
    "stddev",
    "median",
    "averageweight",
    "owned",
    "trading",
    "wanting",
    "wishing",
    "numcomments",
    "numweights",
    "numratings",
    "suggested_playerage",
    "suggested_playerage_quartiles",
    "language_dependence",
    "language_dependence_quartiles",
    "player_count_recs",
    "suggested_numplayers",
    "versions",
    "description",
]

In [29]:
query_time = int(time())
df_parq_merged = df_parq_merged[
    [
        "id",
        "thumbnail",
        "image",
        "description",
        "minplayers",
        "maxplayers",
        "playingtime",
        "minplaytime",
        "maxplaytime",
        "minage",
        "boardgamecategory",
        "boardgamemechanic",
        "boardgamefamily",
        "boardgameexpansion",
        "boardgameartist",
        "boardgamecompilation",
        "boardgameimplementation",
        "boardgamedesigner",
        "boardgamepublisher",
        "boardgameintegration",
        "stddev",
        "median",
        "averageweight",
        "owned",
        "trading",
        "wanting",
        "wishing",
        "numcomments",
        "numweights",
        "numratings",
        "suggested_playerage",
        "suggested_playerage_quartiles",
        "language_dependence",
        "language_dependence_quartiles",
        "player_count_recs",
        "suggested_numplayers",
        "versions",        
    ]
]
df_parq_merged.to_parquet(f"../../data/crawler/boardgame_data_{query_time}.parquet")

In [28]:
df_parq_merged.loc[df_parq_merged["description"].isna()]

,id,thumbnail,image,description,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
16038,17701,https://cf.geekdo-images.com/MpAtPZ7kc5nfxgB7p...,https://cf.geekdo-images.com/MpAtPZ7kc5nfxgB7p...,NaN,2,4,0,0,0,4,"{'1002': 'Card Game', '1041': 'Children's Game'}",{},{'7543': 'Animals: Alligators / Crocodiles'},{},{},{},{},"{'5273': 'R. Blissenbach', '5274': 'I. Gruttma...",{'34': 'Ravensburger'},{},3.50000,0.0,0.0,0,0,0,1,0,0,2,{},{},{},{},"{'Best': ['1', '2', '3', '4', '4+']}","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
43034,64162,None,None,NaN,2,5,30,30,30,13,"{'1002': 'Card Game', '1033': 'Mafia'}","{'2041': 'Open Drafting', '2004': 'Set Collect...",{},{},{'33711': 'CW Karstens'},{},{},{'33711': 'CW Karstens'},{'3320': '(Unknown)'},{},0.00000,0.0,2.0,1,0,0,0,1,1,0,{},{},{},{},"{'Best': ['1', '2', '3', '4', '5', '5+']}","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
55228,118047,https://cf.geekdo-images.com/Bum1Wjoh8FtB3B9ZM...,https://cf.geekdo-images.com/Bum1Wjoh8FtB3B9ZM...,NaN,1,1,40,40,40,12,"{'1001': 'Political', '1019': 'Wargame'}","{'2001': 'Action Points', '2018': 'Campaign / ...","{'22185': 'Admin: Cancelled Games', '5666': 'P...",{},{},{},{},"{'10535': 'Darin A. Leviloff', '36053': 'John ...",{'8007': 'Victory Point Games'},{},0.00000,0.0,0.0,1,0,1,12,7,0,0,{},{},{},{},"{'Best': ['1', '1+']}","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
60698,133829,https://cf.geekdo-images.com/_6voBIbjBhDygOHNG...,https://cf.geekdo-images.com/_6voBIbjBhDygOHNG...,NaN,2,5,150,150,150,14,"{'1050': 'Ancient', '1015': 'Civilization', '1...","{'2046': 'Area Movement', '2011': 'Modular Boa...",{'4705': 'Organizations: The Game Artisans of ...,{},{'63673': 'Michael Xuereb'},{},{},{'63673': 'Michael Xuereb'},{'772': '(Looking for a publisher)'},{},0.00000,0.0,3.0,0,0,3,6,0,1,1,{},{},{},{},"{'Best': ['1', '2', '3', '4', '5', '5+']}","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
71130,164541,https://cf.geekdo-images.com/wHdQjPruemBGJfzBK...,https://cf.geekdo-images.com/wHdQjPruemBGJfzBK...,NaN,2,2,10,10,10,5,"{'1009': 'Abstract Strategy', '1002': 'Card Ga...",{'2040': 'Hand Management'},{'61979': 'Players: Two-Player Only Games'},{},{},{},{},{},{'8291': 'Korea Boardgames'},{},1.34048,0.0,0.0,8,0,0,0,2,0,4,"{'total_votes': 1, '2': 0, '3': 0, '4': 0, '5'...","{'75 percent': '5', '50 percent': '5', '25 per...","{'total_votes': 1, '66': 1, '67': 0, '68': 0, ...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '2+'], 'Best': ['2']}","{'total_votes': 1, '1': {'Best': 0, 'Recommend...","[{'version_id': 249770, 'width': 4, 'length': ..."
95598,416846,https://cf.geekdo-images.com/R2xE3zDRPllb7pd_w...,https://cf.geekdo-images.com/R2xE3zDRPllb7pd_w...,NaN,2,2,0,0,0,0,{'1042': 'Expansion for Base-game'},{},{},{'268864': 'Undaunted: Normandy'},{},{},{},{},{'29313': 'Osprey Games'},{},1.00000,0.0,0.0,25,0,2,1,1,0,2,{},{},{},{},{},{},None
101385,403211,https://cf.geekdo-images.com/QVcXp-dE0XET19rN_...,https://cf.geekdo-images.com/QVcXp-dE0XET19rN_...,NaN,2,2,240,180,240,14,"{'1019': 'Wargame', '1049': 'World War II'}","{'2046': 'Area Movement', '2072': 'Dice Rollin...","{'22184': 'Admin: Upcoming Releases', '8374': ...",{},{'157066': 'Davide Rossolato'},{},{},{'157065': 'Mirco Corazza'},{'11378': 'Acies Edizioni'},{},0.00000,0.0,0.0,0,0,1,2,0,0,1,{},{},{},{},{},{},"[{'version_id': 684321, 'width': 9, 'length': ..."
103285,406554,https://cf.geekdo-images.com/oztY3D5GK3YoiEKrE...,https://cf.geekdo-images.com/oztY3D5GK3YoiEKrE...,NaN,2,2,0,0,0,4,"

## Get ratings data

In [107]:
import importlib
importlib.reload(crawler)

<module 'crawler.src.crawler' from '/home/ubuntu/git/pax_tt_recommender/crawler/src/crawler.py'>

In [7]:
df_bg_data = pd.read_parquet("../../data/crawler/boardgame_data_1748311285.parquet")
print(df_bg_data.shape)
df_bg_data.head()

(42486, 36)


,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
0,13,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,3,4,120,60,120,10,"[Economic, Negotiation]","[Chaining, Dice Rolling, Hexagon Grid, Income,...","[Animals: Sheep, Components: Hexagonal Tiles, ...","[20 Jahre Darmstadt Spielt, Brettspiel Advents...","[Volkan Baga, Tanja Donner, Pete Fenlon, Jason...","[CATAN 3D Collector's Edition, Catan: 25 Jahre...","[Baden-Württemberg Catan, Catan Geographies: G...",[Klaus Teuber],"[KOSMOS, 64 Ounce Games, 999 Games, Albi, Asmo...",[],1.50101,0.0,2.2871,223336,2306,519,7567,22919,8360,135593,"{'total_votes': 577, '2': 1, '3': 0, '4': 2, '...","{'50 percent': '8', '25 percent': '8', '75 per...","{'total_votes': 346, '1': 21, '2': 294, '3': 3...",{'75 percent': 'Some necessary text - easily m...,"{'Not Recommended': ['1', '2', '4+'], 'Recomme...","{'total_votes': 2071, '1': {'Best': 1, 'Recomm...","[{'version_id': 467634, 'version_nickname': 'A..."
1,822,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,2,5,45,30,45,7,"[Medieval, Territory Building]","[Area Majority / Influence, Enclosure, End Gam...","[Category: Dized Tutorial, Cities: Carcassonne...","[20 Jahre Darmstadt Spielt, Antwerpen (fan exp...","[Marcel Gröber, Doris Matthäus, Anne Pätzke, C...","[Carcassonne Big Box, Carcassonne Big Box 2, C...","[The Ark of the Covenant, Carcassonne Demo, Ca...",[Klaus-Jürgen Wrede],"[Hans im Glück, 64 Ounce Games, 999 Games, Alb...","[Carcassonne: Wheel of Fortune, Mists over Car...",1.31104,0.0,1.8880,209535,2034,689,10188,22592,8466,134541,"{'total_votes': 766, '2': 2, '3': 1, '4': 12, ...","{'25 percent': '6', '75 percent': '8', '50 per...","{'total_votes': 490, '6': 481, '7': 7, '8': 1,...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '5+'], 'Best': ['2',...","{'total_votes': 2659, '1': {'Best': 7, 'Recomm...","[{'version_id': 31146, 'version_nickname': 'Al..."
2,30549,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,2,4,45,45,45,8,"[Medical, Travel]","[Action Points, Chaining, Contracts, Cooperati...","[Components: Map (Global Scale), Components: M...",[Pandemic: Gen Con 2016 Promos – Z-Force Team ...,"[Josh Cappel, Christian Hanisch, Régis Moulun,...",[],"[Fall of Rome, Iberia, Pandemic Legacy: Season...",[Matt Leacock],"[Z-Man Games, (Unknown), ADC Blackfire Enterta...",[],1.33738,0.0,2.3968,214983,3251,619,11200,20067,6189,131235,"{'total_votes': 608, '2': 0, '3': 0, '4': 1, '...","{'25 percent': '8', '75 percent': '10', '50 pe...","{'total_votes': 341, '11': 10, '12': 279, '13'...",{'75 percent': 'Some necessary text - easily m...,"{'Recommended': ['1', '2', '3'], 'Best': ['4']...","{'total_votes': 2123, '1': {'Best': 170, 'Reco...","[{'version_id': 30851, 'version_nickname': 'Al..."
3,68448,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,2,7,30,30,30,10,"[Ancient, Card Game, City Building, Civilizati...","[Closed Drafting, End Game Bonuses, Hand Manag...","[Ancient: Babylon, Ancient: Egypt, Ancient: Gr...","[7 Wonders: Armada, 7 Wonders: Armada – Siracu...","[Dimitri Chappuis, Miguel Coimbra, Etienne Heb...",[],"[7 Wonders (Second Edition), 7 Wonders Duel, 7...",[Antoine Bauza],"[Repos Production, ADC Blackfire Entertainment...",[],1.27709,0.0,2.3158,149251,1934,985,14419,16874,5414,109297,"{'total_votes': 561, '2': 0, '3': 0, '4': 1, '..."

In [82]:
df_ratings = pd.read_parquet("../../data/crawler/boardgame_ratings_1748644891.parquet")
print(df_ratings.shape)
# df_ratings.head()
# 1420

(7199, 22)


In [85]:
df_bg_data.loc[df_bg_data["game_id"]=='360899', "numratings"]

6273    519
Name: numratings, dtype: int64

In [86]:
df_tmp2 = df_ratings.loc[df_ratings["game_id"]=="360899"].drop(columns=["game_id"]).fillna("")
for col in df_tmp2.columns:
    df_tmp2[col] = df_tmp2[col].apply(len)
df_tmp2.sum(axis=1)

1513    525
dtype: int64

In [122]:
# df_bg_data.loc[(df_bg_data["numratings"]>=520) & (df_bg_data["numratings"]<600)].shape #563
# df_bg_data.loc[(df_bg_data["numratings"]>=600) & (df_bg_data["numratings"]<700)].shape #572
# df_bg_data.loc[(df_bg_data["numratings"]>=700) & (df_bg_data["numratings"]<800)].shape #451
# df_bg_data.loc[(df_bg_data["numratings"]>=800) & (df_bg_data["numratings"]<900)].shape #336
# df_bg_data.loc[(df_bg_data["numratings"]>=900) & (df_bg_data["numratings"]<1000)].shape #323
# df_bg_data.loc[(df_bg_data["numratings"]>=2000) & (df_bg_data["numratings"]<8000)].shape #451
df_bg_data.loc[(df_bg_data["numratings"]>=8000)].shape #451

(650, 36)

In [123]:
df_ratings.shape

(7199, 22)

In [124]:
df_ratings.head()

,game_id,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,1.0,2.5,4.5,3.5,1.5,0.0,0.5
0,179350,"[obifrankenobi, cfarrell]","[Carlos Palomino, Beorne, asiok, Jetseb, Janon...","[Grildensnork, Murdockus, vetinari7878, Rabbit...","[Nico D, woj_settlers, halbert82, Chuckila, di...","[Bryo4, carontissimo, LookAtTheBacon, sillue, ...","[nunovix, Wendigohexe, zyxyv, hedisch, tgbruis...","[Orphebus, Kuula, littlebanshee, monkeykenobi,...","[MIKI1980, dbarbaglia, teamat, PlayThisToday, ...","[s-man, Mottchen, dpkmc, Surya, Pum_, LucaLeon...","[stoneghe, KABIA66, skipper2206, Turbo2000, ka...","[nunoR, liga, Quintious, manucube, avanspronse...","[Faustoxx, the_spy, Olaf78, Racing cards, dtiv...","[nintakin, nikalibi]","[Gio8, BhART, franck1912, jrubin1, jerome53240...",None,None,None,None,None,None,None
1,99808,"[gimmster, saxophonechapa, eduart, DavidMcGreg...","[Randy Cox, hkhutchi, Gyges, Kaelistus, garg, ...","[Jbrady, mmtmr, sfox, Cerebremancer, cdefrisco...","[dhailey, Grildensnork, Justinschaffer, jgoyes...","[Lothartvni, BC Indy, nittany0508, fehrmeister...","[HooeyPooey, kiddynamite, zefquaavius, Bobby4t...","[TundraThunder, JensFL, Menma, Guiannos, Mibao...","[Errante, maydaygames, kingofthegrill, mechani...","[d_hoopster, takras, fringer, jeremycoe, Conne...","[DAZHEL, MikoML, Boots01, Braidrock, nymartin2...","[mtn55, soulyogurt, eiopqrtuwy, theapostlegree...","[kmbx13, b4kabob, prosfilaes, NerdDad, kommand...",None,"[Flambers, CRRXVI, paintime, TheBardGamer]","[paulo, McCrank, Kenig]",[kcayoung],"[Spyzou, Mario Pawlowski, Zepheus, Janon, smil...",None,None,None,None
2,286667,[idontplayco_ops],"[dream spawn, aranvulnep, Janasik, realwizard,...","[raile, jonpurkis, AmandaDesignsGames, StellaH...","[povertidonki, Invicta Bricks, ryansmum2008, s...","[stroutqb22, Elpete, Duke_Fleed, rsj77, in8nir...","[feldfan2014, cabbagekingaf, Innsmouth_Look, j...","[boardingschoolbrandi, acecondor, drulive99, h...","[QuixoticQuixotic, fencedingates, AerynSpaceFa...","[TaijiJohn, Samutom84, ctrosejr, Jim1701, iuga...","[AHforever, ku8843, Wolverutto, warta, JimmyEW...","[Grenade, FrostyMeeple, sorcretic, Renno]","[RichardIngram, cardshark28800, jcadillac, cft...",None,"[matty0226, Little_Rocket_Games, alessandra198...","[ultimoratio, hazered, dakotamassie]",None,[radiofyr309],None,None,None,None
3,170041,"[scps14, Simcsa, danchi, cadhlan]","[imoo, missiloon, Sweetdogpaw, djhill, JohansG...","[StefanvM1, Livefree, leffe dubbel, Drval, woj...","[Acido, crazyclark, takokawa, river911, costel...","[Menma, Femtitala, linkme, pethulhu, Dalkuldar...","[morishita, Hanno, SplitCreek, Molotail, Great...","[rjgomes, MattCNixon, cogo71, Izobretenik, pec...","[pecotti, DeDaan, nivatra, Frontliner, Els76, ...","[tosh_loco, strubs42, gruntsudens, Bresto, epi...","[Gorkx77, deezynah, Wickedheart, s0915579408, ...","[Poeppelhoppers, vikachoo, MariBor]","[NatkaGra, pilgrim152, Tolkana, castor1966, Su...","[Rosiria, Marysinka]","[Noni1702, Racing cards, silenthoch, ThunderWo...",None,None,[Shamragnar],None,None,None,None
4,300993,[MaxRokotansky],"[MrIkerito, Geeken, Giulii, chumy, elchabonque...","[Alesbolmol, RafaGR, ckomiya, josecarloscom, a...","[Erreka, RubenMS, Santycapi, Yorkaeff, maxiscp...","[nick_arg, MissRiuku, fustero, Norg222, Hoja]","[Juan_Aguilera, thisisrhun, Braculaventura, Ev...","[AngPredator, Ninorro, darthway, Jesseaglerock...","[HANJEL, MrCaronte77, pablo24091980, sandraast...","[Misiga, facalrivas, Guanarteme80, Parsifal1, ...","[peterpax, Jowyjokes, Alex_Luka23, Charly81, E...","[bichejote, Acoidan85, herbius, Whirlpool, mru...","[jdhorux, xelerad, PabloJimenezRoman, Zelgadis...","[iwancho, robinson_2017]","[NinyaSiniestro, Cascabel93, David S Serrano, ...",None,None,"[moncsal213, RedFate]",None,None,None,None


### Convert the ratings dataframe into the proper format

In [136]:
df_ratings.shape

(7199, 22)

In [37]:
rows = []
for _, row in df_ratings.head().iterrows():
    for rating in df_ratings.columns:
        users = row[rating]
        game_id = row["game_id"]
        if isinstance(users, list):
            for user in users:
                rows.append((user, game_id, float(rating)))
# Convert to DataFrame
ratings_df = pd.DataFrame(rows, columns=['user', 'game', 'rating'])   


In [70]:
# Step 1: Flatten and explode data
rows = []
for _, row in df_ratings.iterrows():
    for rating in df_ratings.columns:
        users = row[rating]
        game_id = row["game_id"]
        if isinstance(users, list):
            for user in users:
                rows.append((user, game_id, float(rating)))
# Convert to DataFrame
ratings_df = pd.DataFrame(rows, columns=['user', 'game', 'rating'])   

# Count ratings per user
user_counts = ratings_df['user'].value_counts()

# Filter users with at least 3 ratings
valid_users = user_counts[user_counts >= 3].index

# Filter the dataframe
filtered_ratings_df = ratings_df[ratings_df['user'].isin(valid_users)].copy()

# Step 2: Encode users and games as integer IDs
user_encoder = LabelEncoder()
game_encoder = LabelEncoder()

filtered_ratings_df['user_id'] = user_encoder.fit_transform(filtered_ratings_df['user'])
filtered_ratings_df['game_id'] = game_encoder.fit_transform(filtered_ratings_df['game'])

# Step 3: Create sparse matrix
user_ids = filtered_ratings_df['user_id'].values
game_ids = filtered_ratings_df['game_id'].values
ratings = filtered_ratings_df['rating'].values

ratings_sparse_matrix = coo_matrix((ratings, (user_ids, game_ids)))
print(ratings_sparse_matrix.shape)
# del(filtered_ratings_df)
# del(ratings_df)
# del(rows)

(293414, 7199)


In [ ]:
sparse_matrix_csr = ratings_sparse_matrix.tocsr()

In [72]:
item_user_matrix = csr_matrix(ratings_sparse_matrix.T)

model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
model.fit(item_user_matrix)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:52<00:00,  2.62s/it]


In [42]:
[i for i in similar]

[array([  1713,  95151, 254597, 140033, 187336], dtype=int32),
 array([1.        , 0.67304045, 0.6638781 , 0.6574248 , 0.62207943],
       dtype=float32)]

In [78]:
filtered_ratings_df

,user,game,rating,user_id,game_id
0,obifrankenobi,179350,2.0,248946,1713
1,cfarrell,179350,2.0,180811,1713
2,Carlos Palomino,179350,3.0,23059,1713
3,Beorne,179350,3.0,14664,1713
4,asiok,179350,3.0,169540,1713
...,...,...,...,...,...
11191849,Alexandrooo,303553,10.0,4600,4450
11191850,vagabondo,303553,10.0,284213,4450
11191851,ale10011,303553,10.0,165061,4450
11191852,TheTurbopower,303553,1.0,144578,4450


In [75]:
# Pick a game_id (integer, not original string)
game_id = 1713  # example

# Get similar items (game IDs and similarity scores)
similar = model.similar_items(game_id, N=5)

# Decode back to original game names
# similar_games = [(item_encoder.inverse_transform([i])[0], score) for i, score in similar]
similar_games = list(zip(similar[0],similar[1]))
for game, score in similar_games:
    print(f"{game}: similarity {score:.4f}")

1713: similarity 1.0000
95151: similarity 0.7374
140033: similarity 0.6832
145013: similarity 0.6580
92110: similarity 0.6571


In [ ]:
model.similar_items(itemid)

In [ ]:
filtered_ratings_df["game"]

In [81]:
filtered_ratings_df.loc[filtered_ratings_df["user_id"]==140033]

,user,game,rating,user_id,game_id
1202395,TCPlayer,369898,6.0,140033,5605
7203123,TCPlayer,260428,6.0,140033,3589
7542620,TCPlayer,156858,7.0,140033,1135
7884565,TCPlayer,286751,7.5,140033,4138
7961151,TCPlayer,301880,6.0,140033,4416


In [79]:
filtered_ratings_df.loc[filtered_ratings_df["game"]=='140033']

,user,game,rating,user_id,game_id


In [19]:
ratings_sparse_matrix.

<COOrdinate sparse matrix of dtype 'float64'
	with 24788 stored elements and shape (19364, 50)>

In [ ]:
# Melt the dataframe to long format
long_format = df_ratings.head(2000).melt(id_vars='game_id', var_name='rating', value_name='users')

# Drop rows where users is empty
long_format = long_format.dropna(subset=['users'])

# Explode the user lists
long_format = long_format.explode('users')

# Convert score to int
long_format['rating'] = long_format['rating'].astype(float)

# Now pivot to user-item matrix
user_item_matrix = long_format.pivot_table(index='users', columns='game_id', values='rating')

del long

In [137]:
user_item_matrix

game_id,1002,102610,103651,103814,10453,104575,104805,105199,10527,108157,110868,11161,11202,112991,11385,113927,117854,118337,1190,119193,12140,121409,121764,123129,12318,124490,124545,1250,125368,12747,128412,128733,12875,1289,129122,129615,130390,130552,130999,132326,13464,135648,135888,136558,137651,137811,137900,1383,139508,140271,140612,1408,1416,143981,143994,144526,144632,144743,146158,146559,146579,147240,147451,147568,147614,148083,150298,15046,1508,151,151446,1518,15261,1527,1533,153728,15410,154510,1555,156457,156840,157088,15712,15738,157912,158100,158564,1593,160418,160419,160744,161138,161880,163576,16362,164031,164237,164354,16444,164702,...,3921,392611,393352,39336,393887,395364,395846,396655,397788,400266,40107,40213,402324,40234,40251,4047,40483,406174,40935,409572,414302,415090,415513,415938,418354,423434,423648,4282,42929,432527,43691,4385,4388,4425,4522,4604,4606,4659,482,4985,5014,5135,5296,55835,5585,55863,5651,5740,58886,63,64,64675,6508,6551,6569,6613,66613,66798,67178,71099,719,7204,723,724,7240,72644,72766,7308,7320,7349,7475,7514,75448,756,7803,8039,8232,82955,83040,86128,88402,89,89342,8944,89668,89962,90474,911,9149,92776,9297,93194,94837,951,9616,9617,983,988,99459,99808
users,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-=Yod@=-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Johnny-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Loren-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Mal-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [235]:
df_bg_data_raw

,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
0,13,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,3,4,120,60,120,10,"[Economic, Negotiation]","[Chaining, Dice Rolling, Hexagon Grid, Income,...","[Animals: Sheep, Components: Hexagonal Tiles, ...","[20 Jahre Darmstadt Spielt, Brettspiel Advents...","[Volkan Baga, Tanja Donner, Pete Fenlon, Jason...","[CATAN 3D Collector's Edition, Catan: 25 Jahre...","[Baden-Württemberg Catan, Catan Geographies: G...",[Klaus Teuber],"[KOSMOS, 64 Ounce Games, 999 Games, Albi, Asmo...",[],1.50101,0.0,2.2871,223336,2306,519,7567,22919,8360,135593,"{'total_votes': 577, '2': 1, '3': 0, '4': 2, '...","{'50 percent': '8', '25 percent': '8', '75 per...","{'total_votes': 346, '1': 21, '2': 294, '3': 3...",{'75 percent': 'Some necessary text - easily m...,"{'Not Recommended': ['1', '2', '4+'], 'Recomme...","{'total_votes': 2071, '1': {'Best': 1, 'Recomm...","[{'version_id': 467634, 'version_nickname': 'A..."
1,822,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,2,5,45,30,45,7,"[Medieval, Territory Building]","[Area Majority / Influence, Enclosure, End Gam...","[Category: Dized Tutorial, Cities: Carcassonne...","[20 Jahre Darmstadt Spielt, Antwerpen (fan exp...","[Marcel Gröber, Doris Matthäus, Anne Pätzke, C...","[Carcassonne Big Box, Carcassonne Big Box 2, C...","[The Ark of the Covenant, Carcassonne Demo, Ca...",[Klaus-Jürgen Wrede],"[Hans im Glück, 64 Ounce Games, 999 Games, Alb...","[Carcassonne: Wheel of Fortune, Mists over Car...",1.31104,0.0,1.8880,209535,2034,689,10188,22592,8466,134541,"{'total_votes': 766, '2': 2, '3': 1, '4': 12, ...","{'25 percent': '6', '75 percent': '8', '50 per...","{'total_votes': 490, '6': 481, '7': 7, '8': 1,...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '5+'], 'Best': ['2',...","{'total_votes': 2659, '1': {'Best': 7, 'Recomm...","[{'version_id': 31146, 'version_nickname': 'Al..."
2,30549,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,2,4,45,45,45,8,"[Medical, Travel]","[Action Points, Chaining, Contracts, Cooperati...","[Components: Map (Global Scale), Components: M...",[Pandemic: Gen Con 2016 Promos – Z-Force Team ...,"[Josh Cappel, Christian Hanisch, Régis Moulun,...",[],"[Fall of Rome, Iberia, Pandemic Legacy: Season...",[Matt Leacock],"[Z-Man Games, (Unknown), ADC Blackfire Enterta...",[],1.33738,0.0,2.3968,214983,3251,619,11200,20067,6189,131235,"{'total_votes': 608, '2': 0, '3': 0, '4': 1, '...","{'25 percent': '8', '75 percent': '10', '50 pe...","{'total_votes': 341, '11': 10, '12': 279, '13'...",{'75 percent': 'Some necessary text - easily m...,"{'Recommended': ['1', '2', '3'], 'Best': ['4']...","{'total_votes': 2123, '1': {'Best': 170, 'Reco...","[{'version_id': 30851, 'version_nickname': 'Al..."
3,68448,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,2,7,30,30,30,10,"[Ancient, Card Game, City Building, Civilizati...","[Closed Drafting, End Game Bonuses, Hand Manag...","[Ancient: Babylon, Ancient: Egypt, Ancient: Gr...","[7 Wonders: Armada, 7 Wonders: Armada – Siracu...","[Dimitri Chappuis, Miguel Coimbra, Etienne Heb...",[],"[7 Wonders (Second Edition), 7 Wonders Duel, 7...",[Antoine Bauza],"[Repos Production, ADC Blackfire Entertainment...",[],1.27709,0.0,2.3158,149251,1934,985,14419,16874,5414,109297,"{'total_votes': 561, '2': 0, '3': 0, '4': 1, '..."

In [224]:
pd.DataFrame({"game_id":df_ranks_out["game_id"].tolist(), "ratings_pulled":df_ranks_len.sum(axis=1).tolist()})

,game_id,ratings_pulled
0,13,1000
1,822,1000
2,30549,1000
3,68448,1000
4,167791,1000


In [217]:
df_bg_list.loc[df_bg_list["id"].isin([int(x) for x in batch_ids]), "usersrated"]

0    52069
1    55585
2    52425
3    64869
4    26083
Name: usersrated, dtype: int64

In [216]:
id_list = [int(x) for x in batch_ids[:1]]
math.ceil(df_bg_list.loc[df_bg_list["id"].isin([int(x) for x in batch_ids]), "usersrated"].max()/10)

6487

In [194]:
batch_ids

['224517', '161936', '342942', '174430', '233078']

In [40]:
cookies = get_driver_and_cookies()
df_bg_list = get_bg_list(cookies=cookies)

In [187]:
bg_list = get_boardgame_data(boardgame_ids=df_bg_list["id"])

In [155]:
bg_list

[{'game_id': '224517',
  'thumbnail': 'https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__thumb/img/o18rjEemoWaVru9Y2TyPwuIaRfE=/fit-in/200x150/filters:strip_icc()/pic3490053.jpg',
  'image': 'https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__original/img/FpyxH41Y6_ROoePAilPNEhXnzO8=/0x0/filters:format(jpeg)/pic3490053.jpg',
  'minplayers': '2',
  'maxplayers': '4',
  'playingtime': '120',
  'minplaytime': '60',
  'maxplaytime': '120',
  'minage': '14',
  'boardgamecategory': ['Age of Reason',
   'Economic',
   'Industry / Manufacturing',
   'Post-Napoleonic',
   'Trains',
   'Transportation'],
  'boardgamemechanic': ['Chaining',
   'Hand Management',
   'Income',
   'Loans',
   'Market',
   'Network and Route Building',
   'Tags',
   'Tech Trees / Tech Tracks',
   'Turn Order: Stat-Based',
   'Variable Set-up'],
  'boardgamefamily': ['Cities: Birmingham (England)',
   'Components: Map (Regional scale)',
   'Components: Multi-Use Cards',
   'Country: England',
   'Crowdfunding: Ki

In [6]:
5

5

In [191]:
df = pd.DataFrame(bg_list)
df.head()

,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions,ratings
0,224517,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,2,4,120,60,120,14,"[Age of Reason, Economic, Industry / Manufactu...","[Chaining, Hand Management, Income, Loans, Mar...","[Cities: Birmingham (England), Components: Map...",[],"[Gavan Brown, Lina Cossette, David Forest, Gui...",[],[Brass: Lancashire],"[Gavan Brown, Matt Tolman, Martin Wallace]","[Roxley, Arclight Games, Board Game Rookie, Bo...",[],1.42002,0.0,3.8681,74038,261,1758,19908,7168,2524,"{'total_votes': 181, '2': 1, '3': 0, '4': 0, '...","{'25 percent': '12', '75 percent': '14', '50 p...","{'total_votes': 55, '1': 52, '2': 3, '3': 0, '...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '4+'], 'Recommended'...","{'total_votes': 1257, '1': {'Best': 0, 'Recomm...","[{'version_id': 523778, 'version_nickname': 'C...","{1.0: ['bernstein82', 'jyh19930302', 'nandasto..."
1,161936,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,2,4,60,60,60,13,"[Environmental, Medical]","[Action Points, Cooperative Game, Hand Managem...","[Components: Map (Global Scale), Components: M...",[],"[Chris Quilliams, Chris Quilliams, Chris Quill...",[],"[Pandemic Legacy: Season 2, Pandemic]","[Rob Daviau, Matt Leacock]","[Z-Man Games, Asterion Press, Devir, Filosofia...",[],1.60133,0.0,2.8299,86707,498,794,14355,8313,1511,"{'total_votes': 201, '2': 1, '3': 1, '4': 0, '...","{'25 percent': '10', '75 percent': '12', '50 p...","{'total_votes': 95, '6': 1, '7': 0, '8': 9, '9...",{'75 percent': 'Extensive use of text - massiv...,"{'Not Recommended': ['1', '4+'], 'Recommended'...","{'total_votes': 887, '1': {'Best': 21, 'Recomm...","[{'version_id': 329714, 'version_nickname': 'C...","{1.0: ['ionas', 'ionas', 'okstatefan5945', 'ju..."
2,342942,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,1,4,150,90,150,14,"[Animals, Economic, Environmental]","[Action Queue, End Game Bonuses, Grid Coverage...","[Animals: Okapi, Components: Hexagonal Tiles, ...","[Ark Nova: Marine Worlds, Ark Nova: Promotion ...","[Steffen Bieker, Loïc Billiau, Dennis Lohausen...",[],[],[Mathias Wigge],"[Feuerland Spiele, Capstone Games, CMON Global...",[],1.39247,0.0,3.7815,76747,386,1057,15614,6934,2741,"{'total_votes': 290, '2': 0, '3': 0, '4': 0, '...","{'50 percent': '12', '25 percent': '12', '75 p...","{'total_votes': 73, '11': 0, '12': 0, '13': 25...",{'25 percent': 'Moderate in-game text - needs ...,"{'Recommended': ['1', '3'], 'Best': ['2'], 'No...","{'total_votes': 2085, '1': {'Best': 206, 'Reco...","[{'version_id': 623699, 'version_nickname': 'C...","{1.0: ['Der_Rentner', 'sykano', 'mxke', 'b31o4..."


In [192]:
df["ratings"].iloc[0]

{1.0: ['bernstein82',
  'jyh19930302',
  'nandastone',
  'foolio82',
  'SteoanK',
  'big stone',
  'Mystic_Mouse',
  'ingedico',
  'Zyzzyvas',
  'Digid',
  'wroten121',
  'Scottturnbull',
  'Kurisu2fil',
  'FlyD',
  'motaotie',
  'kosherx',
  'regius',
  'Boltana',
  'Boltana',
  'McJarvis',
  'McJarvis',
  'KOMMEPCAHT',
  'sgosaric',
  'z371177175',
  'z371177175',
  'Divcha',
  'Rikdom',
  'Onizukaahhh',
  'str1anger',
  'edsonpb',
  'christhomas226',
  'johnweldy',
  'lumatt',
  'foofire',
  'tinybugprincess',
  'evancheah',
  'hiramjivago',
  'multiks2389',
  'renji252',
  'seabreezesg1',
  'Ssam6',
  'gamerdoge',
  'Martinezijd',
  'p_vokinsark',
  'Skeleton233',
  'Skeleton233',
  'Dornos',
  'Zharik',
  'chairmanhiro',
  'DJBuro',
  'Jacogzvigo',
  'Snoogans79',
  'frejalj',
  'mido1421',
  'JGHarber',
  'Timbo47',
  'BeyonRe',
  'vecnalives',
  'F0ster7',
  'bigmop',
  'Re_Z',
  'iBud',
  'scottcoody',
  'Dyukan',
  'Griffinedwards',
  'pdruzik',
  'Pedro39',
  'Lovesxp001',
  

In [52]:
soup_xml = get_boardgame_info(boardgame_ids=df_bg_list["id"].astype(str).tolist())

In [222]:
batch_ids

['224517', '161936', '342942', '174430', '233078']

In [227]:
# batch_ids = boardgame_ids[batch_num * batch_size : (batch_num + 1) * batch_size]
# batch_ids = [str(x) for x in batch_ids]
batch_ids = ['224517', '161936', '342942', '174430', '233078', '447122']
bg_info_url = f"https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame&stats=1&versions=1&ratingcomments=1&pagesize=10&page=1&id={','.join(batch_ids)}"
bgg_response = requests.get(bg_info_url)
soup_xml = BeautifulSoup(bgg_response.content, "xml")
games = soup_xml.find_all("item", attrs={"type": "boardgame"})

In [225]:
boardgame_ids = df_bg_list["id"].astype(str).tolist()
batch_size = 5
for batch_num in range(math.ceil(len(xml_ids) / batch_size)):
    batch_ids = boardgame_ids[batch_num * batch_size : (batch_num + 1) * batch_size]
    batch_ids = [str(x) for x in batch_ids]
    bg_info_url = f"https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame&stats=1&versions=1&ratingcomments=1&pagesize=10&page=1&id={','.join(batch_ids)}"
    bgg_response = requests.get(bg_info_url)
    soup_xml = BeautifulSoup(bgg_response.content, "xml")
    games = soup_xml.find_all("item", attrs={"type": "boardgame"})
    print(len(games))
    break

5


# Ratings

In [1]:
def fetch_ratings_for_game(game_id, max_pages=1, delay=2):
    """
    Fetch all user ratings for a specific game_id from BGG.
    Returns a list of dicts with game_id, username, and rating.
    """
    page = 1
    all_ratings = []

    while True:
        url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&comments=1&page={page}"
        response = requests.get(url)

        while response.status_code == 202:
            print(
                f"[{game_id}] Waiting for BGG server to prepare comments... Retrying in 3s."
            )
            time.sleep(3)
            response = requests.get(url)

        if response.status_code != 200:
            print(f"[{game_id}] Error: {response.status_code}")
            break

        soup = BeautifulSoup(response.content, "xml")
        comments = soup.find_all("comment")

        if not comments:
            break  # No more comments/pages

        for comment in comments:
            username = comment.get("username")
            rating = comment.get("rating")

            if rating and rating != "N/A":
                all_ratings.append(
                    {
                        "game_id": int(game_id),
                        "username": username,
                        "rating": float(rating),
                    }
                )

        print(f"[{game_id}] Fetched page {page} with {len(comments)} comments.")
        page += 1

        if max_pages and page > max_pages:
            break

        time.sleep(delay)  # Be polite to the BGG server

    return all_ratings

In [8]:
224517
rtngs_dict = fetch_ratings_for_game(game_id=224517)

[224517] Fetched page 1 with 100 comments.


In [24]:
xml_ids = ["224517", "161936"]
xml_ids = ["224517"]
",".join(xml_ids)

'224517'

In [35]:
121//10

12

In [32]:
# xml_ids = ["224517", "161936"]
xml_ids = ["161936"]
pg_num = 1
url = f"https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame&ratingcomments=1&pagesize=100&page={pg_num}&id={','.join(xml_ids)}"
response = requests.get(url)
soup_xml = BeautifulSoup(response.content, "xml")

In [28]:
item_list = []
items = soup_xml.find_all("item")
# for idx, item in enumerate(items):
#     item_list.append(extract_item(item, game_links[idx]))

In [ ]:
def game_data():
    xml_bs = "https://www.boardgamegeek.com/xmlapi2/thing?type=boardgame&stats=1&ratingcomments=1&page=1&pagesize=10&id="
    all_items = []
    for pg in range(1, 51):
        pg_items = []
        ct = 0
        soup_pg = browse_games(pg)
        pg_ids, pg_links = extract_game_ids(soup_pg)
        print(f"page number {pg} attempt number {ct}")
        while len(pg_items) != 100 and ct < 20:
            xml_fl = requests.get(f'{xml_bs}{",".join(pg_ids)}')
            soup_xml = BeautifulSoup(xml_fl.content, "xml")
            pg_items = extract_xml(soup_xml, pg_links)
            ct += 1
            if ct > 1:
                print(f"page number {pg} attempt number {ct}")
                print(len(pg_items))
        all_items += pg_items
    return all_items

def browse_games(page_num):
    bs_url = "https://boardgamegeek.com/browse/boardgame/page/"
    pg_url = f"{bs_url}{page_num}"
    if page_num <= 10:
        pg = requests.get(pg_url)
    else:
        pg = requests.get(pg_url, cookies)
    soup = BeautifulSoup(pg.content, "html.parser")
    return soup

def extract_game_ids(soup):
    bs_pg = "https://boardgamegeek.com/"
    all_games = soup.find_all("td", {"class": "collection_objectname"})
    game_ids = [x.find("a")["href"].split("/")[-2] for x in all_games]
    game_pages = [urljoin(bs_pg, x.find("a")["href"]) for x in all_games]
    return game_ids, game_pages

In [40]:
df

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.40162,8.57697,51930,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.36424,8.51651,55543,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
2,342942,Ark Nova,2021,3,8.34519,8.53540,52254,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
3,174430,Gloomhaven,2017,4,8.32618,8.56402,64832,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.22948,8.57751,26038,0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165280,446755,Massive Darkness 2: Table Leg Promo Card (MVM),2025,0,0.00000,0.00000,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165281,446756,Resafa: MvM Promo Card,2025,0,0.00000,0.00000,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165282,446759,Operation: McDonald's Happy Meal,2022,0,0.00000,0.00000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165283,446760,静寂または音楽 (Silence or Music),2024,0,0.00000,0.00000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
aa = soup.find("div", {"id": "maincontent"})("a")[0]["href"]

In [4]:
df_ratings = pd.read_parquet("../../data/crawler/boardgame_ratings_1748670263.parquet")
df_ratings.shape

(7459, 22)

In [10]:
# df_ratings = df_ratings.rename(columns={"game_id":"id"})
df_ratings["id"] = df_ratings["id"].astype(int)

In [90]:
df_bg_data_2 = pd.read_parquet("../../data/crawler/boardgame_data_1749420286.parquet")

In [93]:
df_bg_data_2.loc[df_bg_data_2["id"]==13,"numratings"]

52    135852
Name: numratings, dtype: int64

In [94]:
df_bg_data.loc[df_bg_data["id"]==13,"numratings"]

52    135689
Name: numratings, dtype: int64

In [11]:
df_ratings.to_parquet("../../data/crawler/boardgame_ratings_1748670264.parquet")

In [37]:
boardgame_data = df_bg_data.loc[df_bg_data["numratings"] > 100].sort_values(
        by="numratings", ascending=False
    )
boardgame_ids = boardgame_data["id"].tolist()
len(boardgame_ids)
# 127594
# 21062

21062

In [41]:
boardgame_ratings= df_ratings.copy()
df_ratings_len = boardgame_ratings.copy()
df_ratings_len = df_ratings_len.drop(columns=["id"])
df_ratings_len = df_ratings_len.fillna("")
for col in df_ratings_len.columns:
    df_ratings_len[col] = df_ratings_len[col].apply(len)
df_ratings_pulled = pd.DataFrame(
    {
        "id": boardgame_ratings["id"].tolist(),
        "ratings_pulled": df_ratings_len.sum(axis=1).tolist(),
    }
)
boardgame_data = boardgame_data.merge(df_ratings_pulled, on="id", how="left")

In [67]:

completed_ids = boardgame_data.loc[
    (boardgame_data["ratings_pulled"] - boardgame_data["numratings"])/(boardgame_data["numratings"])>=-0.01,
    "id",
].tolist()
boardgame_ids = list(set(boardgame_ids).difference(set(completed_ids)))
df_missing_ratings = boardgame_data.loc[
    (boardgame_data["ratings_pulled"] - boardgame_data["numratings"])/(boardgame_data["numratings"])<-0.01
]

In [70]:
ratings_count_dict = pd.Series(
                df_missing_ratings["numratings"].values,
                index=df_missing_ratings["id"],
            ).to_dict()

In [73]:
import math

In [74]:
math.ceil(max(ratings_count_dict.values()) / 100)

1357

In [69]:
df_missing_ratings.shape

(209, 38)

In [75]:
134000/100

1340.0

In [84]:
boardgame_ratings = boardgame_ratings.loc[~(boardgame_ratings["id"].isin(df_missing_ratings["id"]))]

In [85]:
boardgame_ratings.shape

(7250, 22)

In [86]:
df_missing_ratings.shape

(209, 38)

In [87]:
df_ratings.shape

(7459, 22)

In [68]:
df_missing_ratings.head()

,id,thumbnail,image,description,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions,ratings_pulled
0,13,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,"In CATAN (formerly The Settlers of Catan), pla...",3,4,120,60,120,10,"{'1021': 'Economic', '1026': 'Negotiation'}","{'2956': 'Chaining', '2072': 'Dice Rolling', '...","{'7481': 'Animals: Sheep', '67874': 'Component...","{'167903': '20 Jahre Darmstadt Spielt', '17865...","{'12036': 'Volkan Baga', '12382': 'Tanja Donne...","{'17419': 'CATAN 3D Collector's Edition', '298...","{'123386': 'Baden-Württemberg Catan', '38749':...",{'11': 'Klaus Teuber'},"{'37': 'KOSMOS', '24874': '64 Ounce Games', '2...",{},1.50097,0.0,2.2870,223559,2304,520,7567,22932,8362,135689,"{'total_votes': 577, '2': 1, '3': 0, '4': 2, '...","{'50 percent': '8', '25 percent': '8', '75 per...","{'total_votes': 346, '6': 21, '7': 294, '8': 3...",{'75 percent': 'Some necessary text - easily m...,"{'Not Recommended': ['1', '2', '4+'], 'Recomme...","{'total_votes': 2071, '1': {'Best': 1, 'Recomm...","[{'version_id': 467634, 'width': 12, 'length':...",134000.0
5,266192,https://cf.geekdo-images.com/yLZJCVLlIx4c7eJEW...,https://cf.geekdo-images.com/yLZJCVLlIx4c7eJEW...,"Wingspan is&nbsp;a competitive, medium-weight,...",1,5,70,40,70,10,"{'1089': 'Animals', '1002': 'Card Game', '1094...","{'2689': 'Action Queue', '2072': 'Dice Rolling...","{'45672': 'Animals: Birds', '7657': 'Animals: ...",{'339017': 'Frogmouth Fan Pack (fan expansion ...,"{'113749': 'Ana Maria Martinez Jaramillo', '11...",{},"{'436126': 'Finspan', '410201': 'Wyrmspan'}",{'111338': 'Elizabeth Hargrave'},"{'23202': 'Stonemaier Games', '267': '999 Game...",{'366161': 'Wingspan Asia'},1.36861,0.0,2.4715,163576,841,1295,20374,13812,3434,104355,"{'total_votes': 462, '2': 0, '3': 0, '4': 0, '...","{'25 percent': '8', '75 percent': '10', '50 pe...","{'total_votes': 104, '61': 0, '62': 0, '63': 2...",{'25 percent': 'Moderate in-game text - needs ...,"{'Recommended': ['1', '2', '4'], 'Best': ['3']...","{'total_votes': 2009, '1': {'Best': 110, 'Reco...","[{'version_id': 471202, 'width': 12, 'length':...",91100.0
6,173346,https://cf.geekdo-images.com/zdagMskTF7wJBPjX7...,https://cf.geekdo-images.com/zdagMskTF7wJBPjX7...,In many ways 7 Wonders Duel resembles its pare...,2,2,30,30,30,10,"{'1050': 'Ancient', '1002': 'Card Game', '1029...","{'2875': 'End Game Bonuses', '2902': 'Income',...","{'27524': 'Ancient: Babylon', '72535': 'Ancien...","{'309116': '7 Wonders Duel: Agora', '348601': ...",{'12016': 'Miguel Coimbra'},{},{'421006': 'The Lord of the Rings: Duel for Mi...,"{'9714': 'Antoine Bauza', '1727': 'Bruno Catha...","{'4384': 'Repos Production', '23043': 'ADC Bla...",{},1.19718,0.0,2.2244,171778,1297,863,11590,13410,3231,102562,"{'total_votes': 390, '2': 0, '3': 0, '4': 0, '...","{'25 percent': '8', '75 percent': '10', '50 pe...","{'total_votes': 120, '91': 111, '92': 9, '93':...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '2+'], 'Best': ['2']}","{'total_votes': 1432, '1': {'Best': 13, 'Recom...","[{'version_id': 374197, 'width': 8, 'length': ...",97200.0
11,169786,https://cf.geekdo-images.com/7k_nOxpO9OGIjhLq2...,https://cf.geekdo-images.com/7k_nOxpO9OGIjhLq2...,It is a time of unrest in 1920s Europa. The as...,1,5,115,90,115,14,"{'1021': 'Economic', '1046': 'Fighting', '1016...","{'2080': 'Area Majority / Influence', '2857': ...","{'106514': 'Category: Autonomous simulation', ...",{'276229': 'Scythe: Bonus Promo Pack – 6 Promo...,{'33148'

In [49]:
df_ratings_pulled.loc[df_ratings_pulled["id"]==266192]

,id,ratings_pulled
7439,266192,91100


In [55]:
len(df_ratings.loc[df_ratings["id"]==13, "2.0"].iloc[0])

677

In [96]:
df_ratings.shape

(7459, 22)

In [98]:
df_ratings.head()

,id,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,1.0,2.5,4.5,3.5,1.5,0.0,0.5
0,179350,"[obifrankenobi, cfarrell]","[Carlos Palomino, Beorne, asiok, Jetseb, Janon...","[Grildensnork, Murdockus, vetinari7878, Rabbit...","[Nico D, woj_settlers, halbert82, Chuckila, di...","[Bryo4, carontissimo, LookAtTheBacon, sillue, ...","[nunovix, Wendigohexe, zyxyv, hedisch, tgbruis...","[Orphebus, Kuula, littlebanshee, monkeykenobi,...","[MIKI1980, dbarbaglia, teamat, PlayThisToday, ...","[s-man, Mottchen, dpkmc, Surya, Pum_, LucaLeon...","[stoneghe, KABIA66, skipper2206, Turbo2000, ka...","[nunoR, liga, Quintious, manucube, avanspronse...","[Faustoxx, the_spy, Olaf78, Racing cards, dtiv...","[nintakin, nikalibi]","[Gio8, BhART, franck1912, jrubin1, jerome53240...",None,None,None,None,None,None,None
1,99808,"[gimmster, saxophonechapa, eduart, DavidMcGreg...","[Randy Cox, hkhutchi, Gyges, Kaelistus, garg, ...","[Jbrady, mmtmr, sfox, Cerebremancer, cdefrisco...","[dhailey, Grildensnork, Justinschaffer, jgoyes...","[Lothartvni, BC Indy, nittany0508, fehrmeister...","[HooeyPooey, kiddynamite, zefquaavius, Bobby4t...","[TundraThunder, JensFL, Menma, Guiannos, Mibao...","[Errante, maydaygames, kingofthegrill, mechani...","[d_hoopster, takras, fringer, jeremycoe, Conne...","[DAZHEL, MikoML, Boots01, Braidrock, nymartin2...","[mtn55, soulyogurt, eiopqrtuwy, theapostlegree...","[kmbx13, b4kabob, prosfilaes, NerdDad, kommand...",None,"[Flambers, CRRXVI, paintime, TheBardGamer]","[paulo, McCrank, Kenig]",[kcayoung],"[Spyzou, Mario Pawlowski, Zepheus, Janon, smil...",None,None,None,None
2,286667,[idontplayco_ops],"[dream spawn, aranvulnep, Janasik, realwizard,...","[raile, jonpurkis, AmandaDesignsGames, StellaH...","[povertidonki, Invicta Bricks, ryansmum2008, s...","[stroutqb22, Elpete, Duke_Fleed, rsj77, in8nir...","[feldfan2014, cabbagekingaf, Innsmouth_Look, j...","[boardingschoolbrandi, acecondor, drulive99, h...","[QuixoticQuixotic, fencedingates, AerynSpaceFa...","[TaijiJohn, Samutom84, ctrosejr, Jim1701, iuga...","[AHforever, ku8843, Wolverutto, warta, JimmyEW...","[Grenade, FrostyMeeple, sorcretic, Renno]","[RichardIngram, cardshark28800, jcadillac, cft...",None,"[matty0226, Little_Rocket_Games, alessandra198...","[ultimoratio, hazered, dakotamassie]",None,[radiofyr309],None,None,None,None
3,170041,"[scps14, Simcsa, danchi, cadhlan]","[imoo, missiloon, Sweetdogpaw, djhill, JohansG...","[StefanvM1, Livefree, leffe dubbel, Drval, woj...","[Acido, crazyclark, takokawa, river911, costel...","[Menma, Femtitala, linkme, pethulhu, Dalkuldar...","[morishita, Hanno, SplitCreek, Molotail, Great...","[rjgomes, MattCNixon, cogo71, Izobretenik, pec...","[pecotti, DeDaan, nivatra, Frontliner, Els76, ...","[tosh_loco, strubs42, gruntsudens, Bresto, epi...","[Gorkx77, deezynah, Wickedheart, s0915579408, ...","[Poeppelhoppers, vikachoo, MariBor]","[NatkaGra, pilgrim152, Tolkana, castor1966, Su...","[Rosiria, Marysinka]","[Noni1702, Racing cards, silenthoch, ThunderWo...",None,None,[Shamragnar],None,None,None,None
4,300993,[MaxRokotansky],"[MrIkerito, Geeken, Giulii, chumy, elchabonque...","[Alesbolmol, RafaGR, ckomiya, josecarloscom, a...","[Erreka, RubenMS, Santycapi, Yorkaeff, maxiscp...","[nick_arg, MissRiuku, fustero, Norg222, Hoja]","[Juan_Aguilera, thisisrhun, Braculaventura, Ev...","[AngPredator, Ninorro, darthway, Jesseaglerock...","[HANJEL, MrCaronte77, pablo24091980, sandraast...","[Misiga, facalrivas, Guanarteme80, Parsifal1, ...","[peterpax, Jowyjokes, Alex_Luka23, Charly81, E...","[bichejote, Acoidan85, herbius, Whirlpool, mru...","[jdhorux, xelerad, PabloJimenezRoman, Zelgadis...","[iwancho, robinson_2017]","[NinyaSiniestro, Cascabel93, David S Serrano, ...",None,None,"[moncsal213, RedFate]",None,None,None,None


In [100]:
for col in df_rats.columns[1:]:
    print(len(df_rats.loc[df_rats["id"]==13, col].iloc[0]))

678
1793
3740
8861
945
19674
4482
32683
3100
21349
2371
11803
558
7475
314
37
288
100
5


TypeError: object of type 'NoneType' has no len()

In [95]:
df_rats =pd.read_parquet("../../data/crawler/boardgame_ratings_1749423525.parquet")
df_rats.shape

(8158, 22)

In [97]:
df_rats.head()

,id,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,1.0,2.5,4.5,3.5,1.5,0.0,0.5
0,179350,"[obifrankenobi, cfarrell]","[Carlos Palomino, Beorne, asiok, Jetseb, Janon...","[Grildensnork, Murdockus, vetinari7878, Rabbit...","[Nico D, woj_settlers, halbert82, Chuckila, di...","[Bryo4, carontissimo, LookAtTheBacon, sillue, ...","[nunovix, Wendigohexe, zyxyv, hedisch, tgbruis...","[Orphebus, Kuula, littlebanshee, monkeykenobi,...","[MIKI1980, dbarbaglia, teamat, PlayThisToday, ...","[s-man, Mottchen, dpkmc, Surya, Pum_, LucaLeon...","[stoneghe, KABIA66, skipper2206, Turbo2000, ka...","[nunoR, liga, Quintious, manucube, avanspronse...","[Faustoxx, the_spy, Olaf78, Racing cards, dtiv...","[nintakin, nikalibi]","[Gio8, BhART, franck1912, jrubin1, jerome53240...",None,None,None,None,None,None,None
1,99808,"[gimmster, saxophonechapa, eduart, DavidMcGreg...","[Randy Cox, hkhutchi, Gyges, Kaelistus, garg, ...","[Jbrady, mmtmr, sfox, Cerebremancer, cdefrisco...","[dhailey, Grildensnork, Justinschaffer, jgoyes...","[Lothartvni, BC Indy, nittany0508, fehrmeister...","[HooeyPooey, kiddynamite, zefquaavius, Bobby4t...","[TundraThunder, JensFL, Menma, Guiannos, Mibao...","[Errante, maydaygames, kingofthegrill, mechani...","[d_hoopster, takras, fringer, jeremycoe, Conne...","[DAZHEL, MikoML, Boots01, Braidrock, nymartin2...","[mtn55, soulyogurt, eiopqrtuwy, theapostlegree...","[kmbx13, b4kabob, prosfilaes, NerdDad, kommand...",None,"[Flambers, CRRXVI, paintime, TheBardGamer]","[paulo, McCrank, Kenig]",[kcayoung],"[Spyzou, Mario Pawlowski, Zepheus, Janon, smil...",None,None,None,None
2,286667,[idontplayco_ops],"[dream spawn, aranvulnep, Janasik, realwizard,...","[raile, jonpurkis, AmandaDesignsGames, StellaH...","[povertidonki, Invicta Bricks, ryansmum2008, s...","[stroutqb22, Elpete, Duke_Fleed, rsj77, in8nir...","[feldfan2014, cabbagekingaf, Innsmouth_Look, j...","[boardingschoolbrandi, acecondor, drulive99, h...","[QuixoticQuixotic, fencedingates, AerynSpaceFa...","[TaijiJohn, Samutom84, ctrosejr, Jim1701, iuga...","[AHforever, ku8843, Wolverutto, warta, JimmyEW...","[Grenade, FrostyMeeple, sorcretic, Renno]","[RichardIngram, cardshark28800, jcadillac, cft...",None,"[matty0226, Little_Rocket_Games, alessandra198...","[ultimoratio, hazered, dakotamassie]",None,[radiofyr309],None,None,None,None
3,170041,"[scps14, Simcsa, danchi, cadhlan]","[imoo, missiloon, Sweetdogpaw, djhill, JohansG...","[StefanvM1, Livefree, leffe dubbel, Drval, woj...","[Acido, crazyclark, takokawa, river911, costel...","[Menma, Femtitala, linkme, pethulhu, Dalkuldar...","[morishita, Hanno, SplitCreek, Molotail, Great...","[rjgomes, MattCNixon, cogo71, Izobretenik, pec...","[pecotti, DeDaan, nivatra, Frontliner, Els76, ...","[tosh_loco, strubs42, gruntsudens, Bresto, epi...","[Gorkx77, deezynah, Wickedheart, s0915579408, ...","[Poeppelhoppers, vikachoo, MariBor]","[NatkaGra, pilgrim152, Tolkana, castor1966, Su...","[Rosiria, Marysinka]","[Noni1702, Racing cards, silenthoch, ThunderWo...",None,None,[Shamragnar],None,None,None,None
4,300993,[MaxRokotansky],"[MrIkerito, Geeken, Giulii, chumy, elchabonque...","[Alesbolmol, RafaGR, ckomiya, josecarloscom, a...","[Erreka, RubenMS, Santycapi, Yorkaeff, maxiscp...","[nick_arg, MissRiuku, fustero, Norg222, Hoja]","[Juan_Aguilera, thisisrhun, Braculaventura, Ev...","[AngPredator, Ninorro, darthway, Jesseaglerock...","[HANJEL, MrCaronte77, pablo24091980, sandraast...","[Misiga, facalrivas, Guanarteme80, Parsifal1, ...","[peterpax, Jowyjokes, Alex_Luka23, Charly81, E...","[bichejote, Acoidan85, herbius, Whirlpool, mru...","[jdhorux, xelerad, PabloJimenezRoman, Zelgadis...","[iwancho, robinson_2017]","[NinyaSiniestro, Cascabel93, David S Serrano, ...",None,None,"[moncsal213, RedFate]",None,None,None,None


In [83]:
'Dorowar' in df_ratings.loc[df_ratings["id"]==13, "10.0"].iloc[0]

True

In [58]:
135689/100

1356.89

In [4]:
df = pd.read_parquet("../../data/crawler/boardgame_ratings_1749439709.parquet")
df.shape

(16146, 22)

In [7]:
df.iloc[:1000].to_parquet("../../data/crawler/boardgame_ratings_1749439710.parquet")

In [20]:
df = pd.read_csv("../../data/processed/processed_games_data_1749694079.csv", sep="|", escapechar="\\")
df.shape

(128214, 35)

In [14]:
df.shape

(164105, 35)

In [22]:
df.head()

,id,name,thumbnail,image,description,yearpublished,rank,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,owned,trading,wanting,wishing,numcomments,numweights,numratings,bayesaverage,average,stddev,median,averageweight
0,224517,Brass: Birmingham,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,Brass: Birmingham is an economic strategy game...,2018,1.0,52086,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2,4,120,60,120,14,74212,265,1757,19928,7182,2530,52595,8.40146,8.57715,1.41952,0.0,3.8680
1,161936,Pandemic Legacy: Season 1,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,Pandemic Legacy is a co-operative campaign gam...,2015,2.0,55588,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,2,4,60,60,60,13,86772,499,794,14353,8319,1511,56094,8.36386,8.51619,1.60115,0.0,2.8299
2,342942,Ark Nova,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,"In Ark Nova, you will plan and design a modern...",2021,3.0,52459,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,1,4,150,90,150,14,76971,391,1063,15649,6948,2751,52749,8.34620,8.53657,1.39303,0.0,3.7812
3,174430,Gloomhaven,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,Gloomhaven is a game of Euro-inspired tactica...,2017,4.0,64874,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,1,4,120,60,120,14,101172,1182,1165,21423,11310,2669,65274,8.32586,8.56341,1.73711,0.0,3.9142
4,233078,Twilight Imperium: Fourth Edition,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,Twilight Imperium (Fourth Edition) is a game o...,2017,5.0,26087,0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,3,6,480,240,480,14,31447,211,1060,12247,3897,1276,26338,8.22914,8.57682,1.63882,0.0,4.3354
